In [ ]:
# File Authorship Information
__author__ = """Matteo Lulli, Luca Biferale, Giacomo Falcucci, 
                Mauro Sbragaglia, Dong YangL and Xiaowen Shan"""
__copyright__ = """Copyright 2024, Matteo Lulli, Luca Biferale, 
Giacomo Falcucci, Mauro Sbragaglia and Xiaowen Shan, idea.deploy"""
__license__ = """Permission is hereby granted, free of charge, 
to any person obtaining a copy of this software and associated 
documentation files (the "Software"), to deal in the Software 
without restriction, including without limitation the rights to 
use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, 
and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:
The above copyright notice and this permission notice shall be 
included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, 
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES 
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND 
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT 
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER 
DEALINGS IN THE SOFTWARE."""
__maintainer__ = "Matteo Lulli"
__email__ = "matteo.lulli@gmail.com"
__status__ = "Development"

In [ ]:
# Development cell
%load_ext autoreload
%autoreload 2

# Metastable and Unstable Dynamics in multi-phase lattice Boltzmann

Authors: Matteo Lulli (1), Luca Biferale (2), Giacomo Falcucci (3), Mauro Sbragaglia (2), Dong Yang (1) and Xiaowen Shan (1)

(1) Department of Mechanics and Aerospace Engineering, Southern University of Science and Technology, Shenzhen, Guangdong 518055, China

(2) Department of Physics \& INFN, University of Rome \"Tor Vergata\", Via della Ricerca Scientifica 1, 00133, Rome, Italy.

(3) Department of Enterprise Engineering \"Mario Lucertini\", University of Rome \"Tor Vergata\", Via del Politecnico 1, 00133 Rome, Italy; John A. Paulson School of Engineering and Applied Physics, *Harvard University*,  33 Oxford Street, 02138 Cambridge, Massachusetts, USA.

**Abstract:**
We quantitatively characterize the metastability in a multi-phase lattice Boltzmann model. The structure factor of density fluctuations is theoretically obtained and numerically verified to a high precision, for all simulated wave-vectors and reduced temperatures. The static structure factor is found to consistently diverge as the temperature approaches the critical-point or the density approaches the spinodal line at a sub-critical temperature. Theoretically predicted critical exponents are observed in both cases. Finally, the phase separation in the unstable branch follows the same pattern, i.e. the generation of interfaces with different topology, as observed in molecular dynamics simulations. All results can be independently reproduced through the "idea.deploy" framework [https://github.com/lullimat/idea.deploy](https://github.com/lullimat/idea.deploy)

# Reproducibility

This document is intended for those interested readers who want to reproduce the results reported in the paper [https://arxiv.org/abs/2212.07848](https://arxiv.org/abs/2212.07848). In the present case the computational resources needed should be available in general. 

Next development steps will include a class to measure the time required by each cell and output it in a .json file which can be sent to [matteo.lulli@gmail.com](mailto:matteo.lulli@gmail.com) so that average execution times will be available and organized according to the hardware.

Each subsection can be executed independently and reproduce the results which will be stored locally in the directory 'reproduced-data', so that the data will be generated only once.

Plots can also be generated using the same scripts employed for the figures of the paper. Since there are some issues in executing these scripts in the Jupyter environment we include them as separated files which are called from the cells themselves. **In order to reproduce the plots a working 'latex' installation is necessary to be present on the system.**

This file will be kept updated for new local features and developments in the parent project [**idea.deploy**](https://github.com/lullimat/idea.deploy)

This file is supposed to be pulled from the repository [https://github.com/lullimat/arXiv-2212.07848](https://github.com/lullimat/arXiv-2212.07848), from within the "papers" directory the idea.deploy project.

# Paper Results
Using the next subsections/cells it is possible to reproduce all the results and plots reported in the paper.

## Simulations: Hardware Selection
In the following cells you are required to make a choice for the hardware to employ for the simulations. At the moment you need to stick to a given choice in order to visualize all the results. A more elastic management will be provided in future releases.

**These are the only cells in the notebook that have a "global" character, i.e. they are needed by the rest of the notebook for the simulations cell to run.**

By executing (shift-enter) the cell below, the hardware present in your system will be listed

In [ ]:
# Display Hardware (click the arrow to unfold)
import sys
sys.path.append("../../")

from idpy.IdpyCode import IdpyHardware

IdpyHardware()

**To repdroduce the results you need hardware that support OpenCL with 64bits floating point variables (indicated by 'Double :  63') and/or CUDA.**

The variable "preferred_lang" can be set as
- OCL_T for OpenCL
- CUDA_T for CUDA

The variable "preferred_device" need to be set to the number matching the 64-bits floating point precision requirement discussed above.

The variable "preferred_kind" can be used only with OCL_T and can be set as
- "gpu" for selecting GPU devices
- "cpu" for selecting CPU devices

It also possible to run the simulations on 32-bits floating point variables, however the results will not match those presented in the paper.

In [ ]:
# Language imports (click on left arrow to unfold)
import sys
sys.path.append("../../")

from idpy.IdpyCode import CUDA_T, OCL_T, CTYPES_T, idpy_langs_sys

## Figure 1

### Final configuration in the unstable branch

#### Execution Times
1. Apple M1 Max with ```preferred_lang, preferred_device, preferred_kind = OCL_T, 0, "cpu"```
   -- 0d, 1h, 22m

For the following three-dimensional simulations parallel architectures, ```preferred_lang=OCL_T``` or ```preferred_lang=CUDA_T```, should be preferred in order to shorten runtimes.

In [ ]:
preferred_lang, preferred_device, preferred_kind = OCL_T, 0, "gpu"

In [ ]:
# Simulations in the unstable branch for the final configurations

from idpy.LBM.SCFStencils import SCFStencils, BasisVectors
from idpy.LBM.SCThermo import ShanChanEquilibriumCache, ShanChen

_D2E4_P4F4 = SCFStencils(E = BasisVectors(x_max = 2), 
                         len_2s = [1, 2])
_D2E4_P4F4.FindWeights()

def GetDensityUnstablelDistance(a, psi, G, _stencil = _D2E4_P4F4):
    sc_eq_cache = ShanChanEquilibriumCache(stencil = _stencil, 
                                           psi_f = psi, 
                                           G = G, 
                                           c2 = 1/3, 
                                           n_eps = 1e-6)
    _eq_params = sc_eq_cache.GetFromCache()
    
    _sc = ShanChen(psi_f = psi, theta_val = 1/3, G_val = G)
    
    _density = (1 - a) * _sc.extrema[0][0] + a * _sc.extrema[1][0]    
    return _density

import numpy as np

def UsePlotly(lbm, _eq_params, figures_dir, name):
    import plotly.graph_objects as go
    from plotly.figure_factory import create_quiver
    from functools import reduce
    from plotly.subplots import make_subplots
    import matplotlib.pyplot as plt

    _n_swap = lbm.GetDensityField()
    _L = lbm.sims_vars['dim_sizes'][0]

    _width = 800
    X, Y, Z = np.mgrid[0: _L, 0: _L, 0: _L]


    fig = go.Figure(data = [go.Volume(x = X.flatten(), y = Y.flatten(), z = Z.flatten(), 
                                     value = _n_swap.flatten(), 
                                     isomin = _eq_params['n_g'] * 0.999,
                                     isomax = _eq_params['n_l'] * 1.001,
                                     opacity = 0.22, 
                                     colorscale = [(0,"lightblue"), (1,"gray")], 
                                     opacityscale=[[_eq_params['n_g'] * 1., 0], 
                                                   [_eq_params['n_l'], 1]],
                                     showscale = False, surface_count=5)])

    fig.update_layout(scene_xaxis_showticklabels=False,
                      scene_yaxis_showticklabels=False,
                      scene_zaxis_showticklabels=False,
                      scene_xaxis_showline=False,
                      scene_xaxis_title = "", 
                      scene_yaxis_title = "", 
                      scene_zaxis_title = "")

    fig.update_layout(scene = dict(
                        xaxis = dict(
                             backgroundcolor="white",
                             gridcolor="white",
                             showbackground=False,
                             zerolinecolor="white",),
                        yaxis = dict(
                            backgroundcolor="white",
                            gridcolor="white",
                            showbackground=False,
                            zerolinecolor="white"),
                        zaxis = dict(
                            backgroundcolor="white",
                            gridcolor="white",
                            showbackground=False,
                            zerolinecolor="white",),),
                        width = _width, height = int(0.65 * _width), autosize = False, 
                        margin=dict(l=0, r=0, t=0, b=0)
                      )

    fig.write_image(str(figures_dir / (str(name) + '.png')))
    
    plt.figure()
    plt.axis('off')
    _sketch = plt.imread(str(figures_dir / (str(name) + '.png')))
    plt.imshow(_sketch)    
    plt.show()
    plt.close()
    ##fig.show()
    
'''
_b_unstable_values = {'drop': ['\\textbf{(d)}', 0.15], 
                      'dcylinder': ['\\textbf{(dc)}', 0.25], 
                      'flat': ['\\textbf{(f)}', 0.5], 
                      'bcylinder': ['\\textbf{(bc)}', 0.93], 
                      'bubble': ['\\textbf{(b)}', 0.97]}
'''

unstable_As = {'droplet': 0.15,
               'dcylinder': 0.25, 
               'flat': 0.5, 
               'bcylinder': 0.93, 
               'bubble': 0.99}


unstable_seeds = {'droplet': 1, 
                  'dcylinder': 1, 
                  'flat': 1, 
                  'bcylinder': 113, 
                  'bubble': 11117}

'''
Creating the directory for the figures
'''
from pathlib import Path

reproduced_figures = Path("reproduced-figures")
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

'''
Decalring the simulations features
'''
from idpy.LBM.MultiPhase import ShanChenMultiPhase
from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP
from idpy.IdpyStencils.IdpyStencils import IDStencils

_f_stencil = IDStencils['LBM']['SC_D3E4']
_xi_stencil = IDStencils['LBM']['XI_D3Q19']

_L, _G = 96, -3
_kBT = 1e-10
_c_s2, _tau = 1/3, 1

import sympy as sp

n = sp.Symbol('n')
psi_sym = sp.exp(-1/n)
psi_code = 'exp((NType)(-1./ln_0))'

sc_eq_cache = ShanChanEquilibriumCache(stencil = _D2E4_P4F4, 
                                       psi_f = psi_sym, 
                                       G = _G, 
                                       c2 = _c_s2, 
                                       n_eps = 1e-6)

_eq_params = sc_eq_cache.GetFromCache()

'''
Single run timing
'''
from idpy.Utils.SimpleTiming import SimpleTiming
_st = SimpleTiming()

configuration_file_names = []

for configuration_name in list(unstable_As.keys()):
    print(configuration_name)
    output_file = reproduced_figures / (configuration_name + '.png')

    if not output_file.is_file():
        # Declaring simulation object _test_meta
        unstable_sim = \
            ShanChenMultiPhase(dim_sizes = (_L, _L, _L), 
                               xi_stencil = _xi_stencil, 
                               f_stencil = _f_stencil,
                               psi_code = psi_code, 
                               psi_sym = psi_sym, 
                               e2_val = 1, 
                               SC_G = _G, tau = _tau,
                               lang = preferred_lang, device = preferred_device, 
                               cl_kind = preferred_kind, 
                               optimizer_flag = True, 
                               fluctuations = 'Gross2011', 
                               prng_distribution = 'gaussian', 
                               indep_gaussian = False, 
                               prng_kind = 'MMIX',
                               prng_init_from = 'numpy', 
                               init_seed = unstable_seeds[configuration_name])
    
    
        _a = unstable_As[configuration_name]
        _n_start = GetDensityUnstablelDistance(_a, psi_sym, _G)
    
        unstable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                     width = _L//2, direction = 0)
    
        _st.Start()
        unstable_sim.MainLoopGross2011SRT(
            time_steps = range(0, 2 ** 14 + 1, 2 ** 11), 
            convergence_functions = [CheckUConvergenceSCMP],
            profiling = False,
            kBT = _kBT, n0 = _n_start
        )
        _st.End()
    
        _st.PrintElapsedTime()
    
        UsePlotly(unstable_sim, _eq_params, reproduced_figures, configuration_name)
        unstable_sim.End()        
    else:
        print("File", output_file, "already exists!")
    
    configuration_file_names += [configuration_name + '.png']

### Phase diagram

In order to use the precomputed cache you can copy it on a new file that will be used as cache
```bash
$ cp SCEqCache-non-reproduced.json SCEqCache.json
```

#### Execution Times
1. Apple M1 Max:
    CPU times: user 44min 45s, sys: 4.19 s, total: 44min 49s
    Wall time: 44min 50s

In [ ]:
# Getting binodal and spinodal curves

'''
Generating coupling constant values linear in log scale:
- exp(2) / 3 = G_c * c_s^2
'''
_Gs_sketch = -np.exp(np.linspace(np.log((np.exp(2) / 3) * 1.001), np.log(1.02 * np.exp(2) / 3), 2 ** 4))
_Gs_sketch = np.append(_Gs_sketch, -np.exp(np.linspace(np.log(-_Gs_sketch[-1]), np.log(3.85), 2 ** 5)))
'''
Adding the value of G used for the unstable branch simulations
'''
_Gs_sketch = np.append(_Gs_sketch, [-3])
_Gs_sketch

from idpy.LBM.SCThermo import ShanChanEquilibriumCache
from idpy.Utils.SimpleTiming import SimpleTiming

_st = SimpleTiming()

_n_g_binodal, _n_l_binodal, _p_0_binodal = [], [], []
_density_g_spin, _p_g_spin = [], []
_density_l_spin, _p_l_spin = [], []

for _G in _Gs_sketch:
    print("G:", _G)
    print("Getting binodal data...")
    _st.Start()
    '''
    binodal data
    '''
    sc_eq_cache = ShanChanEquilibriumCache(stencil = _D2E4_P4F4, 
                                           psi_f =  psi_sym, 
                                           G = _G, 
                                           c2 = 1/3, 
                                           n_eps = 1e-6)
    _eq_params = sc_eq_cache.GetFromCache()
    print("done!")
    print()
    _st.End()
    _st.PrintElapsedTime()
    
    _n_g_binodal += [_eq_params['n_g']]
    _n_l_binodal += [_eq_params['n_l']]
    _p_0_binodal += [_eq_params['p_0']]
    
    '''
    spinodal data
    '''
    print("Getting spinodal data...")
    _st.Start()    
    _SC = ShanChen(G_val=_G, psi_f=psi_sym, theta_val=1/3, n_eps=1e-6)

    _density_g_spin += [_SC.extrema[0][0]]
    _p_g_spin += [_SC.extrema[0][1]]
    
    _density_l_spin += [_SC.extrema[1][0]]
    _p_l_spin += [_SC.extrema[1][1]]
    print("done!")
    print()
    _st.End()
    _st.PrintElapsedTime()
    
    print("----------------------------------------------")
    print()
    
_n_g_binodal, _n_l_binodal, _p_0_binodal = np.array(_n_g_binodal), np.array(_n_l_binodal), np.array(_p_0_binodal)

_density_g_spin, _p_g_spin = np.array(_density_g_spin), np.array(_p_g_spin)
_density_l_spin, _p_l_spin = np.array(_density_l_spin), np.array(_p_l_spin)

_p_0_binodal_hat_r = _p_0_binodal * (2 * np.exp(2)) / (np.abs(_Gs_sketch))

_p_g_spin_hat_r = np.array(_p_g_spin * (2 * np.exp(2)) / (np.abs(_Gs_sketch)), dtype = np.float64)
_p_l_spin_hat_r = np.array(_p_l_spin * (2 * np.exp(2)) / (np.abs(_Gs_sketch)), dtype = np.float64)

'''
Getting density values in the unstable region
'''
print()
print("Computing density values in the unstable region")
print()
_b_unstable_values = {'drop': ['\\textbf{(d)}', 0.15], 
                      'dcylinder': ['\\textbf{(dc)}', 0.25], 
                      'flat': ['\\textbf{(f)}', 0.5], 
                      'bcylinder': ['\\textbf{(bc)}', 0.93], 
                      'bubble': ['\\textbf{(b)}', 0.97]}

_t_hatm1_r_2 = 1.22

for _type in _b_unstable_values:
    _b = _b_unstable_values[_type][1]
    _n_swap = GetDensityUnstablelDistance(_b, psi_sym, G = _eq_params['G_c'] * _t_hatm1_r_2)
    _b_unstable_values[_type] += [_n_swap]

In [ ]:
# Plotting the phase diagram only
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import scipy.interpolate as interpolate
from idpy.Utils.Plots import SetAxPanelLabelCoords, SetMatplotlibLatexParamas, SetDefaultFonts
from idpy.Utils.Plots import CreateFiguresPanels, SetAxPanelLabel, SetAxTicksFont

from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

SetMatplotlibLatexParamas([rc], [rcParams])
_fonts = SetDefaultFonts([rc])
_fonts['fs'] = 25

#_fonts['ms_large'] = 9
#_fonts['marker_size_large'] = 9

def Pb_hat(n, G):
    return (n / 3 + (G / 3) * (np.exp(-2 / n)) / 2) * 2 * np.exp(2) / (abs(G) / 3)

_n_fine = np.linspace(1e-3, 4, 2 ** 7)

_nx_fig, _ny_fig = 1, 1

_y_size, _xy_ratio = 4.3, 1.2
fig = CreateFiguresPanels(_nx=_nx_fig,_ny=_ny_fig, _x_size=8*_xy_ratio, _y_size=5*_xy_ratio)

_ax_eos = plt.subplot2grid((_ny_fig, _nx_fig), (0, 0), colspan = 1, rowspan = 1)


'''
Plotting isotherms
'''
_pb_hat = lambda n: Pb_hat(n, 3 * _eq_params['G_c'])
_ax_eos.plot(_n_fine, _pb_hat(_n_fine), color = 'black')

##_t_hatm1_r_0 = 1.2
##_pb_hat = lambda n: Pb_hat(n, 3 * _eq_params['G_c'] * _t_hatm1_r_0)
##_ax_eos.plot(_n_fine, _pb_hat(_n_fine), '-.', linewidth = 0.5, color = 'black')

## The lowest isotherm needs to be chopped in five: (i) before metastable, (ii) metastable, (iii) unstable, (iv) metastable, (v) after metastable
_t_hatm1_r_2 = 1.2180175491295142

_pb_hat = lambda n: Pb_hat(n, 3 * _eq_params['G_c'] * _t_hatm1_r_2)

## (i): from _n_fine[0] to _n_g_binodal[-1]
_i_n_raange = np.linspace(_n_fine[0], _n_g_binodal[-1], 2 ** 7)
_ax_eos.plot(_i_n_raange, _pb_hat(_i_n_raange), color = 'black', linewidth=3)

## (ii): from _n_g_binodal[-1] to _density_g_spin[-1]
_ii_n_raange = np.linspace(_n_g_binodal[-1], _density_g_spin[-1], 2 ** 7)
_ax_eos.plot(_ii_n_raange, _pb_hat(_ii_n_raange), color = 'blue', linewidth=3)

## (iii): from _density_g_spin[-1] to _density_l_spin[-1]
_iii_n_raange = np.linspace(_density_g_spin[-1], _density_l_spin[-1], 2 ** 7)
_ax_eos.plot(_iii_n_raange, _pb_hat(_iii_n_raange), color = 'red', linewidth=3)

## (iv): from _density_l_spin[-1] to _n_l_binodal[-1]
_iv_n_raange = np.linspace(_density_l_spin[-1], _n_l_binodal[-1], 2 ** 7)
_ax_eos.plot(_iv_n_raange, _pb_hat(_iv_n_raange), color = 'blue', linewidth=3)

## (v): from _n_l_binodal[-1] to _n_fine[-1]
_v_n_raange = np.linspace(_n_l_binodal[-1], _n_fine[-1], 2 ** 7)
_ax_eos.plot(_v_n_raange, _pb_hat(_v_n_raange), color = 'black', linewidth=3)


'''
_b_unstable_values = {'drop': ['\\textbf{(d)}', 0.15], 
                      'dcylinder': ['\\textbf{(dc)}', 0.25], 
                      'flat': ['\\textbf{(f)}', 0.5], 
                      'bcylinder': ['\\textbf{(bc)}', 0.94], 
                      'bubble': ['\\textbf{(b)}', 0.95]}
'''

for _type in _b_unstable_values:
    _n_swap = _b_unstable_values[_type][2]

    _x_pos, _y_pos=_n_swap, _pb_hat(_n_swap)    

    if _type == 'drop':
        _x_pos=_n_swap * 0.96
        _y_pos=_pb_hat(_n_swap) + 0.05

    if _type == 'dcylinder':
        _x_pos=_n_swap * 0.91
        _y_pos=_pb_hat(_n_swap) - 0.075

    if _type == 'flat':
        _x_pos=_n_swap * 0.93
        _y_pos=_pb_hat(_n_swap) - 0.075

    if _type == 'bcylinder':
        _x_pos=_n_swap * 0.8
        _y_pos=_pb_hat(_n_swap) - 0.06

    if _type == 'bubble':
        _x_pos=_n_swap * 1.02
        _y_pos=_pb_hat(_n_swap) + 0.06

    ## The points corresponding to the different interface topologies

    _ax_eos.plot([_n_swap], [_pb_hat(_n_swap)], marker = 'o', fillstyle='none', color = 'red', zorder=2 ** 7,  markersize=_fonts['ms_small'])
    SetAxPanelLabelCoords(ax=_ax_eos, label=_b_unstable_values[_type][0], fs=_fonts['fs'], 
                          x_pos=_x_pos, y_pos=_y_pos, color='red')

        
'''
Spinodal lines:
need to interpolate the spinodal lines for the color fill
'''
_p_g_spin_hat_r_spl = interpolate.UnivariateSpline(np.flip(_density_g_spin[:-1]), np.flip(_p_g_spin_hat_r[:-1]), s = 1, k = 5)
_p_l_spin_hat_r_spl = interpolate.UnivariateSpline(_density_l_spin[:-1], _p_l_spin_hat_r[:-1], s = 1, k = 5)

'''
Use the interpolation for the plot and shading:
1. find the extrema for the density
'''
from scipy.optimize import bisect

_n_g_min_spin = bisect(lambda n: _p_g_spin_hat_r_spl(n) - _p_0_binodal_hat_r[:-1][-1], 3e-1, 1)
_n_l_max_spin = bisect(lambda n: _p_l_spin_hat_r_spl(n) - _p_0_binodal_hat_r[:-1][-1], 1, 3)

##print(_n_g_min_spin, _n_l_max_spin)
_n_spin_hat_g_fine = np.exp(np.linspace(np.log(_n_g_min_spin), np.log(1), 2 ** 7))
_n_spin_hat_l_fine = np.exp(np.linspace(np.log(1), np.log(_n_l_max_spin), 2 ** 7))[1:]

_n_spin_hat_fine = np.append(_n_spin_hat_g_fine, _n_spin_hat_l_fine)
_p_spin_hat_r = np.append(_p_g_spin_hat_r_spl(_n_spin_hat_g_fine), _p_l_spin_hat_r_spl(_n_spin_hat_l_fine))

_ax_eos.plot(_n_spin_hat_fine, _p_spin_hat_r, '--', color = 'red', linewidth=3)
_ax_eos.fill_between(_n_spin_hat_fine, _p_spin_hat_r, _p_0_binodal_hat_r[:-1][-1], color = 'red', alpha = 0.2)
#_ax_eos.plot(_density_g_spin[:-1], _p_g_spin_hat_r[:-1], '--', color = 'red')
#_ax_eos.plot(_density_l_spin[:-1], _p_l_spin_hat_r[:-1], '--', color = 'red')

_ax_eos.hlines(y = _p_g_spin_hat_r[-1], xmin=_density_g_spin[-1], xmax=4, 
               linestyle = '-.', color = 'red', linewidth = 1.5)
_ax_eos.hlines(y = _p_l_spin_hat_r[-1], xmin=_density_l_spin[-1], xmax=4, 
               linestyle = '-.', color = 'red', linewidth = 1.5)

SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{P}_{\\mbox{\\small{G,SPIN}}}$', fs=_fonts['fs'], 
                      x_pos=4.1, y_pos=_p_g_spin_hat_r[-1] * (1), color='red')
SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{P}_{\\mbox{\\small{L,SPIN}}}$', fs=_fonts['fs'], 
                      x_pos=4.1, y_pos=_p_l_spin_hat_r[-1] * (1), color='red')


'''
Binodal lines:
Need to interpolate the binodal lines for the color fill
Plotting a straight line in blue indicating the coexistence pressure
'''
_p_0_binodal_hat_r_g_spl = interpolate.UnivariateSpline(np.flip(_n_g_binodal[:-1]), np.flip(_p_0_binodal_hat_r[:-1]), s = 1, k = 5)
_p_0_binodal_hat_r_l_spl = interpolate.UnivariateSpline(_n_l_binodal[:-1], _p_0_binodal_hat_r[:-1], s = 1, k = 5)

'''
Gathering the binodal data for the shading later
'''

_n_g_min_bin = bisect(lambda n: _p_0_binodal_hat_r_g_spl(n) - _p_0_binodal_hat_r[:-1][-1], 2e-1, 1)
_n_l_max_bin = bisect(lambda n: _p_0_binodal_hat_r_l_spl(n) - _p_0_binodal_hat_r[:-1][-1], 1, 4)
##print(_n_g_min_bin, _n_l_max_bin)

_ax_eos.plot(_n_g_binodal[:-1], _p_0_binodal_hat_r[:-1], '--', color = 'blue', linewidth=3)
_ax_eos.plot(_n_l_binodal[:-1], _p_0_binodal_hat_r[:-1], '--', color = 'blue', linewidth=3)
_ax_eos.hlines(y = _p_0_binodal_hat_r[-1], xmin=_n_g_binodal[-1], xmax=4, 
               linestyle = '-.', color = 'blue', linewidth = 1.5)
SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{P}_{0}$', fs=_fonts['fs'], 
                      x_pos=4.1, y_pos=_p_0_binodal_hat_r[-1] * (1 - 0.05), color='blue')


##_ax_eos.axhline(y = 1, linestyle = '-.', color = 'red', linewidth = 1.5)

'''
Metastable region shading
1. need to merge two linspaces 2 ** 7 + 2 ** 7 and make a linspace 2 ** 8 for the binodal
'''
_from_bin_to_spin_n_g = np.linspace(_n_g_min_bin, _n_g_min_spin, 2 ** 7)
_bin_shade_gas_n_fine = np.append(_from_bin_to_spin_n_g, _n_spin_hat_g_fine)
_bin_shade_gas_p_fine = np.append(np.full(2 ** 7, _p_0_binodal_hat_r[:-1][-1]), 
                                  _p_g_spin_hat_r_spl(_n_spin_hat_g_fine))

_ax_eos.fill_between(_bin_shade_gas_n_fine, 
                     _p_0_binodal_hat_r_g_spl(_bin_shade_gas_n_fine), 
                     _bin_shade_gas_p_fine, color = 'blue', alpha = 0.2)

_from_bin_to_spin_n_l = np.linspace(_n_l_max_spin, _n_l_max_bin, 2 ** 7)
_bin_shade_liq_n_fine = np.append(_n_spin_hat_l_fine, _from_bin_to_spin_n_l)
_bin_shade_liq_p_fine = np.append(_p_l_spin_hat_r_spl(_n_spin_hat_l_fine), 
                                  np.full(2 ** 7, _p_0_binodal_hat_r[:-1][-1]))

_ax_eos.fill_between(_bin_shade_liq_n_fine, 
                     _p_0_binodal_hat_r_l_spl(_bin_shade_liq_n_fine), 
                     _bin_shade_liq_p_fine, color = 'blue', alpha = 0.2)

'''
Critical point
'''
_ax_eos.plot([1], [1], marker = 'o', color = 'red', markersize=_fonts['ms_small'])

'''
End spinodal points
'''
_ax_eos.plot([_density_g_spin[-1]], [_p_g_spin_hat_r[-1]], marker='o', color = 'red', markersize=_fonts['ms_small'])
_ax_eos.plot([_density_l_spin[-1]], [_p_l_spin_hat_r[-1]], marker='o', color = 'red', markersize=_fonts['ms_small'])

_ax_eos.vlines(x = _density_g_spin[-1], ymin=-1, ymax=_p_g_spin_hat_r[-1], color = 'red', 
               linestyle = '-.', linewidth = 1.5)
_ax_eos.vlines(x = _density_l_spin[-1], ymin=-1, ymax=_p_l_spin_hat_r[-1], color = 'red', 
               linestyle = '-.', linewidth = 1.5)

SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{n}_{g}^{\\mbox{\\small{SPIN}}}$', fs=_fonts['fs'], 
                      x_pos=_density_g_spin[-1] * (1 - 0.02), y_pos=_p_0_binodal_hat_r[:-1][-1] * (1 - 0.175), color='red')
SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{n}_{l}^{\\mbox{\\small{SPIN}}}$', fs=_fonts['fs'], 
                      x_pos=_density_l_spin[-1] * (1 - 0.02), y_pos=_p_0_binodal_hat_r[:-1][-1] * (1 - 0.175), color='red')

'''
End binodal points
'''
_ax_eos.plot([_n_g_binodal[-1]], [_p_0_binodal_hat_r[-1]], marker='o', color = 'blue', markersize=_fonts['ms_small'])
_ax_eos.plot([_n_l_binodal[-1]], [_p_0_binodal_hat_r[-1]], marker='o', color = 'blue', markersize=_fonts['ms_small'])

_ax_eos.vlines(x = _n_g_binodal[-1], ymin=-1, ymax=_p_0_binodal_hat_r[-1], color = 'blue', 
               linestyle = '-.', linewidth = 1.5)
_ax_eos.vlines(x = _n_l_binodal[-1], ymin=-1, ymax=_p_0_binodal_hat_r[-1], color = 'blue', 
               linestyle = '-.', linewidth = 1.5)

SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{n}_{g}^{\\mbox{\\small{BIN}}}$', fs=_fonts['fs'], 
                      x_pos=_n_g_binodal[-1] * (1 - 0.02), y_pos=_p_0_binodal_hat_r[:-1][-1] * (1 - 0.175), color='blue')
SetAxPanelLabelCoords(ax=_ax_eos, label='$\\hat{n}_{l}^{\\mbox{\\small{BIN}}}$', fs=_fonts['fs'], 
                      x_pos=_n_l_binodal[-1] * (1 - 0.02), y_pos=_p_0_binodal_hat_r[:-1][-1] * (1 - 0.175), color='blue')


## Relative temperature labels for each isotherm
SetAxPanelLabel(ax=_ax_eos, label='$\hat{T} = 1$', fs=_fonts['fs'], x_pos=0.52, y_pos=0.925)
SetAxPanelLabel(ax=_ax_eos, label='$\hat{T} =' + str('%.1f' % (1/_t_hatm1_r_2)) + '$', 
                fs=_fonts['fs'], x_pos=0.7, y_pos=0.76)

'''
Axes labels & ticks
'''
_ax_eos.set_xlabel("scaled density $\hat{n}$", fontsize = _fonts['fs'], labelpad=20)
_ax_eos.set_ylabel("scaled pressure $\hat{P}$", fontsize = _fonts['fs'], labelpad=20)

SetAxTicksFont(_ax_eos, _fonts['fs'])

'''
Shadings legend
'''
if False:
    _w_frame, _h_frame = 0.25, 0.125
    _x_offset, _y_offset = 0.01, 0.8

    _ax_eos.fill([_x_offset, _x_offset + _w_frame, 
                  _x_offset + _w_frame, _x_offset], 
                 [_y_offset, _y_offset, 
                  _y_offset + _h_frame, 
                  _y_offset + _h_frame], 
                 'red', transform=_ax_eos.transAxes, 
                 alpha=0.2, edgecolor='black', linewidth=5)

    SetAxPanelLabel(ax=_ax_eos, label='\\textbf{instability}', fs=_fonts['fs'], 
                    x_pos=_x_offset * (1 + 3.5), y_pos=_y_offset + _h_frame * (1 - 0.7), 
                    color='red')

    _w_frame, _h_frame = 0.25, 0.125
    _x_offset, _y_offset = 0.01, 0.6

    _ax_eos.fill([_x_offset, _x_offset + _w_frame, 
                  _x_offset + _w_frame, _x_offset], 
                 [_y_offset, _y_offset, 
                  _y_offset + _h_frame, 
                  _y_offset + _h_frame], 
                 'blue', transform=_ax_eos.transAxes, 
                 alpha=0.2, edgecolor='black', linewidth=5)

    SetAxPanelLabel(ax=_ax_eos, label='\\textbf{metastability}', fs=_fonts['fs'], 
                    x_pos=_x_offset * (1 + 0.85), y_pos=_y_offset + _h_frame * (1 - 0.7), 
                    color='blue')


'''
Axes range
'''
_ax_eos.set_xscale('log')
_ax_eos.set_xlim([2e-1,4])
_ax_eos.set_ylim([_p_0_binodal_hat_r[:-1][-1],1.1])
_ax_eos.set_yticks(np.arange(0.4, 1.1, 0.3))
#_ax_eos.set_ylim([-0.3,1.5])

plt.savefig(reproduced_figures / ('sketch_eos.png'), bbox_inches = 'tight', dpi = 300)
plt.savefig(reproduced_figures / ('sketch_eos.pdf'), bbox_inches = 'tight', dpi = 300)

In [ ]:
# Cropping unstable branch figures

# Cropping the image created in the previous cell (click on left arrow to unfold)
## https://www.kite.com/python/examples/3034/pil-crop-a-region-of-an-image
from PIL import Image
from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

    
for name in unstable_As.keys():
    _im = Image.open(str(reproduced_figures / (name + '.png')))

    _left = (_im.size[0] - _im.size[1]) // 2
    _right = _left + _im.size[1]
    _top = _im.size[1]
    _bottom = _im.size[1] // 9

    _region = _im.crop((_left, _bottom, _right, _top))
    _region.save(str(reproduced_figures / (name + '_cropped.png')))

In [ ]:
# Making the collage and obtaining Figure 1
'''
Collage
'''
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import scipy.interpolate as interpolate
from idpy.Utils.Plots import SetAxPanelLabelCoords

from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

SetMatplotlibLatexParamas([rc], [rcParams])
_fonts = SetDefaultFonts([rc])

_nx_fig, _ny_fig = 6, 9
## _nx_fig, _ny_fig = 15, 8

_scale = 3
_x_size, _y_size = 4 * 4.3 / _scale, 3 * 4.3 / _scale
fig = CreateFiguresPanels(_nx=_nx_fig,_ny=_ny_fig, _x_size = _x_size, _y_size = _y_size)


'''
Equation of state sketch
'''
_ax_eos = plt.subplot2grid((_ny_fig, _nx_fig), (4, 0), colspan = 6, rowspan = 5)

_sketch = plt.imread(str(reproduced_figures / 'sketch_eos.png'))

_ax_eos.imshow(_sketch)
_ax_eos.axis("off")

cropped_files = [name + '_cropped.png' for name in unstable_As.keys()]

'''
Inerface sketches
'''
_ax_interface = plt.subplot2grid((_ny_fig, _nx_fig), (0, 0), colspan = 2, rowspan = 2)
_sketch = plt.imread(str(reproduced_figures / cropped_files[0]))
_ax_interface.imshow(_sketch)
_ax_interface.axis("off")
SetAxPanelLabel(ax=_ax_interface, label='\\textbf{(d)}', fs=_fonts['fs'] * 3.2, x_pos=0.7, y_pos=0.105, color='red')


_ax_interface = plt.subplot2grid((_ny_fig, _nx_fig), (0, 2), colspan = 2, rowspan = 2)
_sketch = plt.imread(str(reproduced_figures / cropped_files[1]))
_ax_interface.imshow(_sketch)
_ax_interface.axis("off")
SetAxPanelLabel(ax=_ax_interface, label='\\textbf{(dc)}', fs=_fonts['fs'] * 3.2, x_pos=0.7, y_pos=0.105, color='red')

_ax_interface = plt.subplot2grid((_ny_fig, _nx_fig), (0, 4), colspan = 2, rowspan = 2)
_sketch = plt.imread(str(reproduced_figures / cropped_files[2]))
_ax_interface.imshow(_sketch)
_ax_interface.axis("off")
SetAxPanelLabel(ax=_ax_interface, label='\\textbf{(f)}', fs=_fonts['fs'] * 3.2, x_pos=0.7, y_pos=0.105, color='red')

_ax_interface = plt.subplot2grid((_ny_fig, _nx_fig), (2, 1), colspan = 2, rowspan = 2)
_sketch = plt.imread(str(reproduced_figures / cropped_files[3]))
_ax_interface.imshow(_sketch)
_ax_interface.axis("off")
SetAxPanelLabel(ax=_ax_interface, label='\\textbf{(bc)}', fs=_fonts['fs'] * 3.2, x_pos=0.7, y_pos=0.105, color='red')

_ax_interface = plt.subplot2grid((_ny_fig, _nx_fig), (2, 3), colspan = 2, rowspan = 2)
_sketch = plt.imread(str(reproduced_figures / cropped_files[4]))
_ax_interface.imshow(_sketch)
_ax_interface.axis("off")
SetAxPanelLabel(ax=_ax_interface, label='\\textbf{(b)}', fs=_fonts['fs'] * 3.2, x_pos=0.7, y_pos=0.105, color='red')

plt.savefig(reproduced_figures / ('figure_1.png'), bbox_inches = 'tight', dpi = 300)
plt.savefig(reproduced_figures / ('figure_1.pdf'), bbox_inches = 'tight', dpi = 300)

## Figure 2

The cell below contains the variable setting the number of dimensions at which to run the simulations. As it is discussed in the paper, an important property of the model is that only one-dimensional calculations are involved. 

Therefore, it is computationally convienient to run all simulations first in $d=1$. Higher dimensions can be set, depending on the available computational resources.

When running with $d=1$ it is suggested to use the the C++ single-thread version, i.e. ```preferred_kind=CTYPES_T```, for larger dimension the user should resort to parallel architectures, i.e. ```preferred_kind=OCL_T``` or ```preferred_kind=CUDA_T``` to minimize the runtime.

In [ ]:
dimension = 1
preferred_lang, preferred_device, preferred_kind = OCL_T, 0, "gpu"

### Simulations

In [ ]:
# Parameters preparation for both Figure 2 and Figure 3

import numpy as np
import sympy as sp

from MetastableUnstableSCAnalysis import ComputeMeanErr
from MetastableUnstableSCAnalysis import StructureFactors3D
from MetastableUnstableSCAnalysis import StructureFactors2D
from MetastableUnstableSCAnalysis import StructureFactors1D
from MetastableUnstableSCAnalysis import StructureFactorsData

'''
Values for the relative distance from the 
'''
_a_fine = 1 - np.exp(np.linspace(np.log(1e-2), np.log(1), 2 ** 5))
_a_fine = np.flip(_a_fine)

_a_fine_G = 1 - np.exp(np.linspace(np.log(1e-5), np.log(1), 2 ** 5))
_a_fine_G = np.flip(_a_fine_G)
_a_fine_G

_G_c = -np.exp(2) / 3
'''
The factor 1.56 has been manually set for tuning the value of G
near the critical temperature
'''
_G_fine = _a_fine_G * _G_c + (1 - _a_fine_G) * (_G_c * (1.56))

'''
Preparing the stencil for the computation of the equilibrium values:
We can use the two dimensional stencil because the Maxwell construction
assumes flat interface profile: in this case three- and two-dimensional
simulations would yield the same result
'''
from idpy.LBM.SCFStencils import SCFStencils, BasisVectors
from idpy.LBM.SCThermo import ShanChanEquilibriumCache, ShanChen

_D2E4_P4F4 = SCFStencils(E = BasisVectors(x_max = 2), 
                         len_2s = [1, 2])
_D2E4_P4F4.FindWeights()

'''
Function to get the density for a gas/liquid homogeneous state given the relative distance between
binodal and spinodal lines at a given temperature
'''
def GetDensitySpinodalDistance(a, psi, G, d_type = 'n_g', _stencil = _D2E4_P4F4):
    if d_type not in ['n_g', 'n_l']:
        raise Exception("Argument 'd_type' must be in ['n_g', 'n_l']")
        
    sc_eq_cache = ShanChanEquilibriumCache(stencil = _stencil, 
                                           psi_f = psi, 
                                           G = G, 
                                           c2 = 1/3, 
                                           n_eps = 1e-6)
    _eq_params = sc_eq_cache.GetFromCache()
    
    _sc = ShanChen(psi_f = psi, theta_val = 1/3, G_val = G, n_eps=1e-6)
    
    _density = 0
    if d_type == 'n_g':
        _density = (1 - a) * _eq_params['n_g'] + a * _sc.extrema[0][0]
    if d_type == 'n_l':
        _density = (1 - a) * _eq_params['n_l'] + a * _sc.extrema[1][0]
    
    return _density

'''
Creating the directory for the figures
'''
from pathlib import Path

reproduced_figures = Path("reproduced-figures")
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

'''
Decalring the simulations features
'''
from idpy.LBM.MultiPhase import ShanChenMultiPhase
from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP
from idpy.IdpyStencils.IdpyStencils import IDStencils

if dimension == 1:
    _f_stencil = IDStencils['LBM']['SC_D1Q2']
    _xi_stencil = IDStencils['LBM']['XI_D1Q3']
    _structure_factor_analysis = StructureFactors1D
    _time_tuple = (0, 2 ** 22 + 1, 2 ** 9)
elif dimension == 2:
    _f_stencil = IDStencils['LBM']['SC_D2E4']
    _xi_stencil = IDStencils['LBM']['XI_D2Q9']
    _structure_factor_analysis = StructureFactors2D
    _time_tuple = (0, 2 ** 20 + 1, 2 ** 9)    
elif dimension == 3:
    _f_stencil = IDStencils['LBM']['SC_D3E4']
    _xi_stencil = IDStencils['LBM']['XI_D3Q19']
    _structure_factor_analysis = StructureFactors3D
    _time_tuple = (0, 2 ** 20 + 1, 2 ** 9)
    
"""
Figure 2(a) indices: _i_a_list = [0, 5, 13, 31]
Figure 2(b) indices: _i_a_list = [0, 9, 17, 31]
Figure 2(c) indices: _g_i_list = [0, 6, 14, 31]
Figure 2(d) indices: _g_i_list = [0, 9, 14, 31]
"""

print()

If you do not wish to re-execute all the simulations you can copy the results from the file provided in the repository by inputting the (bash/zsh) terminal command

```bash

$ cp SFactorsData-non-reproduced.json SFactorsData.json
```

After this the cells below will not run any simulation.
After this step, you can simply go back to the original state by removing the file by inputting the (bash/zsh) terminal command
```bash
$ rm SFactorsData.json
```

#### Execution Times
1. Apple M1 Max with ```preferred_lang, preferred_device, preferred_kind = OCL_T, 0, "cpu"```
   -- 0d, 0h, 9m - $d = 1$

In [ ]:
# Simulations/Data Reading for Fig 2(a)

'''
Data for Fig 2(a)
'''
print("Figure 2(a) data")
if True:
    _L, _tau = 128, 1
    figure_2a_i_a_list, _G, _kBT = [0, 5, 13, 31], -3, 1e-13

    n = sp.Symbol('n')
    psi_sym = sp.exp(-1/n)
    psi_code = 'exp((NType)(-1./ln_0))'

    from idpy.Utils.SimpleTiming import SimpleTiming
    _st = SimpleTiming()

    for seed, a_value in enumerate(_a_fine[figure_2a_i_a_list]):

        _s_data = StructureFactorsData(dim=dimension, L=_L, n_type='n_g', a=a_value, G=_G, kBT=_kBT)
        N_check = _s_data.CheckContent()

        print("Checking if the data is already there...", end="")
        if N_check != (_time_tuple[1] // _time_tuple[2]):
            print("no!")
            print("Running the simulation")
            # Declaring simulation object _test_meta
            matastable_sim = \
                ShanChenMultiPhase(dim_sizes = (_L,) * dimension, 
                                   xi_stencil = _xi_stencil, 
                                   f_stencil = _f_stencil,
                                   psi_code = psi_code, 
                                   psi_sym = psi_sym, 
                                   e2_val = 1, 
                                   SC_G = _G, tau = _tau,
                                   lang = preferred_lang, device = preferred_device, 
                                   cl_kind = preferred_kind, 
                                   optimizer_flag = True, 
                                   fluctuations = 'Gross2011', 
                                   prng_distribution = 'gaussian', 
                                   indep_gaussian = False, 
                                   prng_kind = 'MMIX',
                                   prng_init_from = 'numpy', 
                                   init_seed=seed)

            _n_start = GetDensitySpinodalDistance(a_value, psi_sym, _G, 'n_g')

            matastable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                             width = _L//2, direction = 0)


            _st.Start()
            matastable_sim.MainLoopGross2011SRT(
                time_steps = range(*_time_tuple), 
                convergence_functions = [_structure_factor_analysis],
                profiling = False,
                kBT = _kBT, n0 = _n_start
            )
            _st.End()

            _st.PrintElapsedTime()

            _n2_ft_data = np.array(matastable_sim.sims_vars['n2_ft'])

            _n2_ft_ave, _n2_ft_err = [], []
            for _i in range(_n2_ft_data.shape[1]):
                _mean_swap, _err_swap = ComputeMeanErr(_n2_ft_data[:,_i])    
                _n2_ft_ave += [_mean_swap]
                _n2_ft_err += [_err_swap]    

            _n2_ft_ave, _n2_ft_err = np.array(_n2_ft_ave), np.array(_n2_ft_err)

            _k_range = matastable_sim.sims_vars['nx_list'] * 2 * np.pi / _L    

            _data_dict = \
                {'k_range': _k_range[1:], 'n_start': _n_start, 
                 'Sk': {'ave': _n2_ft_ave[1:], 'err': _n2_ft_err[1:], 
                        'N': len(_n2_ft_data[:,0])}}


            _s_data.PushData(data_dict=_data_dict)

            print("-------------------------------------------------------------------------")
            print()

            matastable_sim.End()
        else:
            print("yes!")

In [ ]:
# Simulations/Data Reading for Fig 2(b)
'''
Data for Fig 2(b)
'''
print("Figure 2(b) data")
if True:
    _L, _tau = 128, 1
    figure_2b_i_a_list, _G, _kBT = [0, 9, 17, 31], -3, 1e-13

    n = sp.Symbol('n')
    psi_sym = sp.exp(-1/n)
    psi_code = 'exp((NType)(-1./ln_0))'

    from idpy.Utils.SimpleTiming import SimpleTiming
    _st = SimpleTiming()

    
    for seed, a_value in enumerate(_a_fine[figure_2b_i_a_list]):

        _s_data = StructureFactorsData(dim=dimension, L=_L, n_type='n_l', a=a_value, G=_G, kBT=_kBT)
        N_check = _s_data.CheckContent()

        print("Checking if the data is already there...", end="")        
        if N_check != (_time_tuple[1] // _time_tuple[2]):
            print("no!")
            print("Running the simulation")            
            # Declaring simulation object _test_meta
            matastable_sim = \
                ShanChenMultiPhase(dim_sizes = (_L,) * dimension, 
                                   xi_stencil = _xi_stencil, 
                                   f_stencil = _f_stencil,
                                   psi_code = psi_code, 
                                   psi_sym = psi_sym, 
                                   e2_val = 1, 
                                   SC_G = _G, tau = _tau,
                                   lang = preferred_lang, device = preferred_device, 
                                   cl_kind = preferred_kind, 
                                   optimizer_flag = True, 
                                   fluctuations = 'Gross2011', 
                                   prng_distribution = 'gaussian', 
                                   indep_gaussian = False, 
                                   prng_kind = 'MMIX',
                                   prng_init_from = 'numpy', 
                                   init_seed=seed)

            _n_start = GetDensitySpinodalDistance(a_value, psi_sym, _G, 'n_l')

            matastable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                             width = _L//2, direction = 0)

            _st.Start()
            matastable_sim.MainLoopGross2011SRT(
                time_steps = range(*_time_tuple), 
                convergence_functions = [_structure_factor_analysis],
                profiling = False,
                kBT = _kBT, n0 = _n_start
            )
            _st.End()

            _st.PrintElapsedTime()

            _n2_ft_data = np.array(matastable_sim.sims_vars['n2_ft'])

            _n2_ft_ave, _n2_ft_err = [], []
            for _i in range(_n2_ft_data.shape[1]):
                _mean_swap, _err_swap = ComputeMeanErr(_n2_ft_data[:,_i])    
                _n2_ft_ave += [_mean_swap]
                _n2_ft_err += [_err_swap]    

            _n2_ft_ave, _n2_ft_err = np.array(_n2_ft_ave), np.array(_n2_ft_err)

            _k_range = matastable_sim.sims_vars['nx_list'] * 2 * np.pi / _L    

            _data_dict = \
                {'k_range': _k_range[1:], 'n_start': _n_start, 
                 'Sk': {'ave': _n2_ft_ave[1:], 'err': _n2_ft_err[1:], 
                        'N': len(_n2_ft_data[:,0])}}


            _s_data.PushData(data_dict=_data_dict)

            print("-------------------------------------------------------------------------")
            print()

            matastable_sim.End()
        else:
            print("yes!")            

In [ ]:
# Simulations/Data Reading for Fig 2(c)
'''
Data for Fig 2(c)
'''
print("Figure 2(c) data")
if True:
    _L, _tau = 128, 1
    figure_2c_g_i_list, _a, _kBT = [0, 6, 14, 31], 0, 1e-13

    n = sp.Symbol('n')
    psi_sym = sp.exp(-1/n)
    psi_code = 'exp((NType)(-1./ln_0))'

    from idpy.Utils.SimpleTiming import SimpleTiming
    _st = SimpleTiming()

    
    for seed, G_value in enumerate(_G_fine[figure_2c_g_i_list]):

        _s_data = StructureFactorsData(dim=dimension, L=_L, n_type='n_g', a=_a, G=G_value, kBT=_kBT)
        N_check = _s_data.CheckContent()

        print("Checking if the data is already there...", end="")        
        if N_check != (_time_tuple[1] // _time_tuple[2]):
            print("no!")
            print("Running the simulation")            
            # Declaring simulation object _test_meta
            matastable_sim = \
                ShanChenMultiPhase(dim_sizes = (_L,) * dimension, 
                                   xi_stencil = _xi_stencil, 
                                   f_stencil = _f_stencil,
                                   psi_code = psi_code, 
                                   psi_sym = psi_sym, 
                                   e2_val = 1, 
                                   SC_G = G_value, tau = _tau,
                                   lang = preferred_lang, device = preferred_device, 
                                   cl_kind = preferred_kind, 
                                   optimizer_flag = True, 
                                   fluctuations = 'Gross2011', 
                                   prng_distribution = 'gaussian', 
                                   indep_gaussian = False, 
                                   prng_kind = 'MMIX',
                                   prng_init_from = 'numpy', 
                                   init_seed=seed)

            _n_start = GetDensitySpinodalDistance(_a, psi_sym, G_value, 'n_g')

            matastable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                             width = _L//2, direction = 0)

            _st.Start()
            matastable_sim.MainLoopGross2011SRT(
                time_steps = range(*_time_tuple), 
                convergence_functions = [_structure_factor_analysis],
                profiling = False,
                kBT = _kBT, n0 = _n_start
            )
            _st.End()

            _st.PrintElapsedTime()

            _n2_ft_data = np.array(matastable_sim.sims_vars['n2_ft'])

            _n2_ft_ave, _n2_ft_err = [], []
            for _i in range(_n2_ft_data.shape[1]):
                _mean_swap, _err_swap = ComputeMeanErr(_n2_ft_data[:,_i])    
                _n2_ft_ave += [_mean_swap]
                _n2_ft_err += [_err_swap]    

            _n2_ft_ave, _n2_ft_err = np.array(_n2_ft_ave), np.array(_n2_ft_err)

            _k_range = matastable_sim.sims_vars['nx_list'] * 2 * np.pi / _L    

            _data_dict = \
                {'k_range': _k_range[1:], 'n_start': _n_start, 
                 'Sk': {'ave': _n2_ft_ave[1:], 'err': _n2_ft_err[1:], 
                        'N': len(_n2_ft_data[:,0])}}


            _s_data.PushData(data_dict=_data_dict)

            print("-------------------------------------------------------------------------")
            print()

            matastable_sim.End()
        else:
            print("yes!")            

In [ ]:
# Simulations/Data Reading for Fig 2(d)
'''
Data for Fig 2(d)
'''
print("Figure 2(d) data")
if True:
    _L, _tau = 128, 1
    figure_2d_g_i_list, _a, _kBT = [0, 9, 14, 31], 0, 1e-13

    n = sp.Symbol('n')
    psi_sym = sp.exp(-1/n)
    psi_code = 'exp((NType)(-1./ln_0))'

    from idpy.Utils.SimpleTiming import SimpleTiming
    _st = SimpleTiming()

    
    for seed, G_value in enumerate(_G_fine[figure_2d_g_i_list]):

        _s_data = StructureFactorsData(dim=dimension, L=_L, n_type='n_l', a=_a, G=G_value, kBT=_kBT)
        N_check = _s_data.CheckContent()

        print("Checking if the data is already there...", end="")        
        if N_check != (_time_tuple[1] // _time_tuple[2]):
            print("no!")
            print("Running the simulation")            
            # Declaring simulation object _test_meta
            matastable_sim = \
                ShanChenMultiPhase(dim_sizes = (_L,) * dimension, 
                                   xi_stencil = _xi_stencil, 
                                   f_stencil = _f_stencil,
                                   psi_code = psi_code, 
                                   psi_sym = psi_sym, 
                                   e2_val = 1, 
                                   SC_G = G_value, tau = _tau,
                                   lang = preferred_lang, device = preferred_device, 
                                   cl_kind = preferred_kind, 
                                   optimizer_flag = True, 
                                   fluctuations = 'Gross2011', 
                                   prng_distribution = 'gaussian', 
                                   indep_gaussian = False, 
                                   prng_kind = 'MMIX',
                                   prng_init_from = 'numpy', 
                                   init_seed=seed)

            _n_start = GetDensitySpinodalDistance(_a, psi_sym, G_value, 'n_l')

            matastable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                             width = _L//2, direction = 0)

            _st.Start()
            matastable_sim.MainLoopGross2011SRT(
                time_steps = range(*_time_tuple), 
                convergence_functions = [_structure_factor_analysis],
                profiling = False,
                kBT = _kBT, n0 = _n_start
            )
            _st.End()

            _st.PrintElapsedTime()

            _n2_ft_data = np.array(matastable_sim.sims_vars['n2_ft'])

            _n2_ft_ave, _n2_ft_err = [], []
            for _i in range(_n2_ft_data.shape[1]):
                _mean_swap, _err_swap = ComputeMeanErr(_n2_ft_data[:,_i])    
                _n2_ft_ave += [_mean_swap]
                _n2_ft_err += [_err_swap]    

            _n2_ft_ave, _n2_ft_err = np.array(_n2_ft_ave), np.array(_n2_ft_err)

            _k_range = matastable_sim.sims_vars['nx_list'] * 2 * np.pi / _L    

            _data_dict = \
                {'k_range': _k_range[1:], 'n_start': _n_start, 
                 'Sk': {'ave': _n2_ft_ave[1:], 'err': _n2_ft_err[1:], 
                        'N': len(_n2_ft_data[:,0])}}


            _s_data.PushData(data_dict=_data_dict)

            print("-------------------------------------------------------------------------")
            print()

            matastable_sim.End()
        else:
            print("yes!")            

In [ ]:
# Collecting data from the json file
'''
Values for the relative distance from the 
'''
_a_fine = 1 - np.exp(np.linspace(np.log(1e-2), np.log(1), 2 ** 5))
_a_fine = np.flip(_a_fine)

_a_fine_G = 1 - np.exp(np.linspace(np.log(1e-5), np.log(1), 2 ** 5))
_a_fine_G = np.flip(_a_fine_G)
_a_fine_G

_G_c = -np.exp(2) / 3
'''
The factor 1.56 has been manually set for tuning the value of G
near the critical temperature
'''
_G_fine = _a_fine_G * _G_c + (1 - _a_fine_G) * (_G_c * (1.56))


'''
Fig 2(a)
'''
_L, _tau = 128, 1
figure_2a_i_a_list, _G, _kBT = [0, 5, 13, 31], -3, 1e-13

_data_spin_ng = []

for a_value in _a_fine[figure_2a_i_a_list]:
    _sf_data = \
        StructureFactorsData(dim=dimension, L=_L, n_type='n_g', a=a_value, G=_G, kBT=_kBT)
    
    _data_spin_ng += [_sf_data.PullData(_sf_data.DataKeyPrefix())]
    
_data_spin_ng = np.array(_data_spin_ng)

'''
Fig 2(b)
'''
_L, _tau = 128, 1
figure_2b_i_a_list, _G, _kBT = [0, 9, 17, 31], -3, 1e-13

_data_spin_nl = []

for a_value in _a_fine[figure_2b_i_a_list]:
    _sf_data = \
        StructureFactorsData(dim=dimension, L=_L, n_type='n_l', a=a_value, G=_G, kBT=_kBT)
    
    _data_spin_nl += [_sf_data.PullData(_sf_data.DataKeyPrefix())]
    
_data_spin_nl = np.array(_data_spin_nl)

'''
Fig 2(c)
'''
_L, _tau = 128, 1
figure_2c_g_i_list, _a, _kBT = [0, 6, 14, 31], 0, 1e-13

_data_bin_ng = []

for G_value in _G_fine[figure_2c_g_i_list]:
    _sf_data = \
        StructureFactorsData(dim=dimension, L=_L, n_type='n_g', a=_a, G=G_value, kBT=_kBT)
    
    _data_bin_ng += [_sf_data.PullData(_sf_data.DataKeyPrefix())]
    
_data_bin_ng = np.array(_data_bin_ng)

'''
Fig 2(d)
'''
_L, _tau = 128, 1
figure_2d_g_i_list, _a, _kBT = [0, 9, 14, 31], 0, 1e-13

_data_bin_nl = []

for G_value in _G_fine[figure_2d_g_i_list]:
    _sf_data = \
        StructureFactorsData(dim=dimension, L=_L, n_type='n_l', a=_a, G=G_value, kBT=_kBT)
    
    _data_bin_nl += [_sf_data.PullData(_sf_data.DataKeyPrefix())]
    
_data_bin_nl = np.array(_data_bin_nl)

In [ ]:
# Generating the curves according to the analytical results

_k_fine_alt = np.linspace(0, np.pi, 2 ** 8)


def Sk2(k, c_s2, Gcs2, n0, kBT, psi_sym):
    _psi_f = sp.lambdify(n, psi_sym)
    _d_psi_f = sp.lambdify(n, psi_sym.diff())
    _psi0 = _psi_f(n0)
    _dpsi0 = _d_psi_f(n0)
    _val = \
        n0 * kBT / \
        (c_s2 + Gcs2 * (_psi0 * _dpsi0) - (k ** 2) * Gcs2 * _psi0 * _dpsi0 / 4)
    return _val

def Sk(k, c_s2, Gcs2, n0, kBT, psi_sym):
    _psi_f = sp.lambdify(n, psi_sym)
    _d_psi_f = sp.lambdify(n, psi_sym.diff())
    _psi0 = _psi_f(n0)
    _dpsi0 = _d_psi_f(n0)
    _val = \
        n0 * kBT / \
        (c_s2 + Gcs2 * (_psi0 * _dpsi0) + Gcs2 * _psi0 * _dpsi0 * (np.cos(k) - 1) / 2)
    return _val

def Sk2Hat(k, T_hat, n0_hat, Theta_hat, psi_sym):
    _psi_f = sp.lambdify(n, psi_sym)
    _d_psi_f = sp.lambdify(n, psi_sym.diff())
    _psi0 = _psi_f(n0_hat)
    _dpsi0 = _d_psi_f(n0_hat)
    _val = \
        2. * n0_hat * Theta_hat / \
        (2. * T_hat - 2. * (_psi0 * _dpsi0) + (k ** 2) * _psi0 * _dpsi0 / 2)
    return _val

def SkHat(k, T_hat, n0_hat, Theta_hat, psi_sym):
    _psi_f = sp.lambdify(n, psi_sym)
    _d_psi_f = sp.lambdify(n, psi_sym.diff())
    _psi0 = _psi_f(n0_hat)
    _dpsi0 = _d_psi_f(n0_hat)
    _val = \
        2. * n0_hat * Theta_hat / \
        (2. * T_hat - 2. * (_psi0 * _dpsi0) - _psi0 * _dpsi0 * (np.cos(k) - 1))
    return _val


psi_sym = sp.exp(-1/n)
psi_sym_hat = sp.exp(1-1/n)

'''
Fig 2(a) theoretical values
'''
_Sk_fig_2a, _Sk_fig_2a_k2 = [], []
_Sk_fig_2a_hat, _Sk_fig_2a_k2_hat = [], []

for a_value in _a_fine[figure_2a_i_a_list]:
    _n_init = GetDensitySpinodalDistance(a_value, psi_sym, _G, d_type='n_g')
    _sk_lambda = lambda k: Sk(k, 1/3, _G, _n_init, _kBT, psi_sym)
    _sk_lambda_k2 = lambda k: Sk2(k, 1/3, _G, _n_init, _kBT, psi_sym)

    T_hat = np.exp(2) / np.abs(_G * 3)
    Theta_hat = _kBT * np.exp(2)
    _sk_lambda_hat = lambda k: SkHat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    _sk_lambda_k2_hat = lambda k: Sk2Hat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    
    print("Gas density:", _n_init)

    _Sk_fig_2a += [[_sk_lambda(_k) for _k in _k_fine_alt]]
    _Sk_fig_2a_k2 += [[_sk_lambda_k2(_k) for _k in _k_fine_alt]]
    _Sk_fig_2a_hat += [[_sk_lambda_hat(_k) for _k in _k_fine_alt]]
    _Sk_fig_2a_k2_hat += [[_sk_lambda_k2_hat(_k) for _k in _k_fine_alt]]

_Sk_fig_2a, _Sk_fig_2a_k2 = \
    np.array(_Sk_fig_2a), np.array(_Sk_fig_2a_k2)
_Sk_fig_2a_hat, _Sk_fig_2a_k2_hat = \
    np.array(_Sk_fig_2a_hat), np.array(_Sk_fig_2a_k2_hat)

'''
Fig 2(b) theoretical values
'''
_Sk_fig_2b, _Sk_fig_2b_k2 = [], []
_Sk_fig_2b_hat, _Sk_fig_2b_k2_hat = [], []

for a_value in _a_fine[figure_2b_i_a_list]:
    _n_init = GetDensitySpinodalDistance(a_value, psi_sym, _G, d_type='n_l')
    _sk_lambda = lambda k: Sk(k, 1/3, _G, _n_init, _kBT, psi_sym)
    _sk_lambda_k2 = lambda k: Sk2(k, 1/3, _G, _n_init, _kBT, psi_sym)

    T_hat = np.exp(2) / np.abs(_G * 3)
    Theta_hat = _kBT * np.exp(2)
    _sk_lambda_hat = lambda k: SkHat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    _sk_lambda_k2_hat = lambda k: Sk2Hat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    
    print("Liquid density:", _n_init)

    _Sk_fig_2b += [[_sk_lambda(_k) for _k in _k_fine_alt]]
    _Sk_fig_2b_k2 += [[_sk_lambda_k2(_k) for _k in _k_fine_alt]]
    _Sk_fig_2b_hat += [[_sk_lambda_hat(_k) for _k in _k_fine_alt]]
    _Sk_fig_2b_k2_hat += [[_sk_lambda_k2_hat(_k) for _k in _k_fine_alt]]    

_Sk_fig_2b, _Sk_fig_2b_k2 = \
    np.array(_Sk_fig_2b), np.array(_Sk_fig_2b_k2)
_Sk_fig_2b_hat, _Sk_fig_2b_k2_hat = \
    np.array(_Sk_fig_2b_hat), np.array(_Sk_fig_2b_k2_hat)

'''
Fig 2(c) theoretical values
'''
_Sk_fig_2c, _Sk_fig_2c_k2 = [], []
_Sk_fig_2c_hat, _Sk_fig_2c_k2_hat = [], []

for G_value in _G_fine[figure_2c_g_i_list]:
    _n_init = GetDensitySpinodalDistance(0, psi_sym, G_value, d_type='n_g')
    _sk_lambda = lambda k: Sk(k, 1/3, G_value, _n_init, _kBT, psi_sym)
    _sk_lambda_k2 = lambda k: Sk2(k, 1/3, G_value, _n_init, _kBT, psi_sym)

    T_hat = np.exp(2) / np.abs(G_value * 3)
    Theta_hat = _kBT * np.exp(2)
    _sk_lambda_hat = lambda k: SkHat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    _sk_lambda_k2_hat = lambda k: Sk2Hat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    
    print("Liquid density:", _n_init)

    _Sk_fig_2c += [[_sk_lambda(_k) for _k in _k_fine_alt]]
    _Sk_fig_2c_k2 += [[_sk_lambda_k2(_k) for _k in _k_fine_alt]]
    _Sk_fig_2c_hat += [[_sk_lambda_hat(_k) for _k in _k_fine_alt]]
    _Sk_fig_2c_k2_hat += [[_sk_lambda_k2_hat(_k) for _k in _k_fine_alt]]

_Sk_fig_2c, _Sk_fig_2c_k2 = \
    np.array(_Sk_fig_2c), np.array(_Sk_fig_2c_k2)
_Sk_fig_2c_hat, _Sk_fig_2c_k2_hat = \
    np.array(_Sk_fig_2c_hat), np.array(_Sk_fig_2c_k2_hat)

'''
Fig 2(d) theoretical values
'''
_Sk_fig_2d, _Sk_fig_2d_k2 = [], []
_Sk_fig_2d_hat, _Sk_fig_2d_k2_hat = [], []

for G_value in _G_fine[figure_2d_g_i_list]:
    _n_init = GetDensitySpinodalDistance(0, psi_sym, G_value, d_type='n_l')
    _sk_lambda = lambda k: Sk(k, 1/3, G_value, _n_init, _kBT, psi_sym)
    _sk_lambda_k2 = lambda k: Sk2(k, 1/3, G_value, _n_init, _kBT, psi_sym)        

    T_hat = np.exp(2) / np.abs(G_value * 3)
    Theta_hat = _kBT * np.exp(2)
    _sk_lambda_hat = lambda k: SkHat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    _sk_lambda_k2_hat = lambda k: Sk2Hat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    
    print("Liquid density:", _n_init)

    _Sk_fig_2d += [[_sk_lambda(_k) for _k in _k_fine_alt]]
    _Sk_fig_2d_k2 += [[_sk_lambda_k2(_k) for _k in _k_fine_alt]]
    _Sk_fig_2d_hat += [[_sk_lambda_hat(_k) for _k in _k_fine_alt]]
    _Sk_fig_2d_k2_hat += [[_sk_lambda_k2_hat(_k) for _k in _k_fine_alt]]

_Sk_fig_2d, _Sk_fig_2d_k2 = \
    np.array(_Sk_fig_2d), np.array(_Sk_fig_2d_k2)
_Sk_fig_2d_hat, _Sk_fig_2d_k2_hat = \
    np.array(_Sk_fig_2d_hat), np.array(_Sk_fig_2d_k2_hat)

'''
Bulk pressure and spinodal pressure for the isotherm \hat{T}=0.8, G=-3
'''

def Pb(n, G):
    return n / 3 + (G) * (np.exp(-2 / n)) / 2

_G = -3
_Pb = lambda n: Pb(n, _G)    

_sc = ShanChen(psi_f = sp.exp(-1/n), theta_val = 1/3, G_val = _G)

_p_spin_g, _p_spin_l = _sc.extrema[0][1], _sc.extrema[1][1]
_G_crit = _sc.G_c

In [ ]:
# Collecting in arrays the data for S(k)
'''
Metastable gas
'''
_2a_sk_spin_g_ave, _2a_sk_spin_g_err, _n_spin_gas = [], [], []
for _i in range(len(_data_spin_ng)):
    _n_spin_gas += [_data_spin_ng[_i]['n_start']]
    _2a_sk_spin_g_ave += [np.array(_data_spin_ng[_i]['Sk']['ave'])]
    _2a_sk_spin_g_err += [np.array(_data_spin_ng[_i]['Sk']['err'])]

_2a_sk_spin_g_ave = np.array(_2a_sk_spin_g_ave)
_2a_sk_spin_g_err = np.array(_2a_sk_spin_g_err)
_n_spin_gas = np.array(_n_spin_gas)

_delta_Ps_gas = (_Pb(_n_spin_gas)) / float(_p_spin_g)

'''
Metastable liquid
'''
_2b_sk_spin_l_ave, _2b_sk_spin_l_err, _n_spin_liq = [], [], []
for _i in range(len(_data_spin_nl)):
    _n_spin_liq += [_data_spin_nl[_i]['n_start']]
    _2b_sk_spin_l_ave += [np.array(_data_spin_nl[_i]['Sk']['ave'])]
    _2b_sk_spin_l_err += [np.array(_data_spin_nl[_i]['Sk']['err'])]

_2b_sk_spin_l_ave = np.array(_2b_sk_spin_l_ave)
_2b_sk_spin_l_err = np.array(_2b_sk_spin_l_err)
_n_spin_liq = np.array(_n_spin_liq)

_delta_Ps_liq = (_Pb(_n_spin_liq)) / float(_p_spin_l)

'''
Stable gas
'''
_2c_sk_spin_g_ave, _2c_sk_spin_g_err, _n_bin_gas = [], [], []
for _i in range(len(_data_bin_ng)):
    _n_bin_gas += [_data_bin_ng[_i]['n_start']]
    _2c_sk_spin_g_ave += [np.array(_data_bin_ng[_i]['Sk']['ave'])]
    _2c_sk_spin_g_err += [np.array(_data_bin_ng[_i]['Sk']['err'])]

_2c_sk_spin_g_ave = np.array(_2c_sk_spin_g_ave)
_2c_sk_spin_g_err = np.array(_2c_sk_spin_g_err)
_n_bin_gas = np.array(_n_bin_gas)

_delta_Ts_gas = float(_G_crit) / _G_fine[figure_2c_g_i_list]
_delta_Ts_gas = 1 - _delta_Ts_gas

'''
Stable liquid
'''
_2d_sk_spin_l_ave, _2d_sk_spin_l_err, _n_bin_liq = [], [], []
for _i in range(len(_data_bin_nl)):
    _n_bin_liq += [_data_bin_nl[_i]['n_start']]
    _2d_sk_spin_l_ave += [np.array(_data_bin_nl[_i]['Sk']['ave'])]
    _2d_sk_spin_l_err += [np.array(_data_bin_nl[_i]['Sk']['err'])]

_2d_sk_spin_l_ave = np.array(_2d_sk_spin_l_ave)
_2d_sk_spin_l_err = np.array(_2d_sk_spin_l_err)
_n_bin_liq = np.array(_n_bin_liq)

_delta_Ts_liq = 1 - float(_G_crit) / _G_fine[figure_2d_g_i_list]

In [ ]:
# Plot: Fig 2 all panels
import math

import matplotlib

from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import scipy.interpolate as interpolate
from idpy.Utils.Plots import SetAxPanelLabelCoords, SetMatplotlibLatexParamas, SetAxTicksFont
from idpy.Utils.Plots import SetDefaultFonts, CreateFiguresPanels, SetAxPanelLabel
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

SetMatplotlibLatexParamas([rc], [rcParams])
_fonts = SetDefaultFonts([rc])
_fonts['fs'] = 24
'''
LatexESpecifier
'''
def LatexESpecifier(n, precision = 2, times = '\\times'):
    _n_c_scientific = '%21.15e' % n
    _n_mantissa = float(_n_c_scientific.split("e")[0])
    _n_exp = int(_n_c_scientific.split("e")[1])
    
    _str_mantissa = ("%." + str(precision) + "f") % _n_mantissa
    _str_exp = str(_n_exp)
    return _str_mantissa + times + " 10^{" + _str_exp + "}"

_alpha_crit = 0.03
_alpha_a = 0.05

_nx_fig, _ny_fig = 2, 2

_ratio = 1.05
fig = CreateFiguresPanels(_nx=_nx_fig,_ny=_ny_fig, _x_size=6.75/_ratio, _y_size=5.8/_ratio)

_one_m_pi_s = "1 - \\hat{\pi}_{\\mbox{\\small{S}}}"
#_one_m_pi_s = '\boldsymbol{\pi}'
_pi_s_m_one = "\\hat{\pi}_{\\mbox{\\small{S}}} - 1"

_ax_sk_meta_g_k = plt.subplot2grid((_ny_fig, _nx_fig), (0, 0), colspan = 1, rowspan = 1)
_panel_label="$(a)$"

if True:
    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_a_fine), vmax = np.amax(_a_fine))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkorange","darkgreen"])
    
    
    SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$\\mathbf{' + _one_m_pi_s + '}$', 
                    fs=_fonts['fs'], x_pos=0.7, y_pos=0.71, color='black')
    
    
    axins1 = inset_axes(_ax_sk_meta_g_k,
                        width="25%",  # width: 50% of parent_bbox width
                        height="5%",  # height: 5%
                        loc="upper right", 
                        bbox_to_anchor=[-.065, -.3, 1, 1],
                        bbox_transform=_ax_sk_meta_g_k.transAxes
                       )
    
    fig.colorbar(cm.ScalarMappable(norm=_gs_col_norm,cmap=_cmap), 
                 cax=axins1, orientation="horizontal", ticks=[])

    '''
    Repeated...?
    '''
    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_a_fine), vmax = np.amax(_a_fine))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkgreen","darkorange"])
    
    
    _k_fine = np.array(_data_spin_ng[0]['k_range'])
        
    _every = 1
    
    _i_a_list = [0, 5, 13, 31]
    ## (GSequence(n_steps=5, num=1, den=1) - 1)
    for index, _i_a in enumerate(_i_a_list):
        _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
        _ax_sk_meta_g_k.plot(_k_fine[::_every], np.abs(_G) * _2a_sk_spin_g_ave[index,:][::_every], 
                             marker='o', linestyle='none', color=_color)

        _ax_sk_meta_g_k.plot(_k_fine_alt, _Sk_fig_2a_hat[index,:], color=_color)
        _ax_sk_meta_g_k.plot(_k_fine_alt, _Sk_fig_2a_k2_hat[index,:], color=_color, linestyle='--')        

    _ax_sk_meta_g_k.set_yscale('log')
    _ax_sk_meta_g_k.set_xscale('log')
    
    _ax_sk_meta_g_k.set_xticks([0, np.pi/32, np.pi/16, np.pi/8, np.pi/4, np.pi/2, np.pi])
    _ax_sk_meta_g_k.set_xticklabels(['0', '$\pi/32$', '$\pi/16$', '$\pi/8$', '$\pi/4$', '$\pi/2$', '$\pi$'])
    '''
    TODO: set the k range to _k_fine[0]
    '''
    _ax_sk_meta_g_k.set_xlim([8e-1 * _k_fine[1], np.pi])
    
    SetAxTicksFont(_ax_sk_meta_g_k, _fonts['fs'])
    
    _ax_sk_meta_g_k.set_ylabel('$\hat{S}(k\Delta x)$', fontsize = _fonts['fs'])
    _ax_sk_meta_g_k.set_xlabel('$k\Delta x$', fontsize = _fonts['fs'])    
    SetAxPanelLabel(ax=_ax_sk_meta_g_k, label=_panel_label, fs=_fonts['fs'], pos='ur')
    _ax_sk_meta_g_k.set_ylim([3.5e-13, 1.5e-10])

    
    _i_a = _i_a_list[3]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = 1 - _delta_Ps_gas[3]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.71, color=_color, rotation=-33)
        
        
    _i_a = _i_a_list[2]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = 1 - _delta_Ps_gas[2]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.47, color=_color, rotation=-5)
        
    _i_a = _i_a_list[1]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = 1 - _delta_Ps_gas[1]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.245, color=_color, rotation=0)
        
    _i_a = _i_a_list[0]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = 1 - _delta_Ps_gas[0]
    _label_value_ltx = LatexESpecifier(_label_value, precision=1, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.075, color=_color, rotation=0)
        
    SetAxPanelLabel(ax=_ax_sk_meta_g_k, label='{Gas} -- ${\\hat{T} = ' + ("%.1f" % (_G_c/_G)) + '}$', 
                    fs=_fonts['fs'], x_pos=0.47, y_pos=0.9)

#############################################################################

_ax_sk_meta_l_k = plt.subplot2grid((_ny_fig, _nx_fig), (0, 1), colspan = 1, rowspan = 1)
_panel_label="$(b)$"

if True:
    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_a_fine), vmax = np.amax(_a_fine))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkorange","darkgreen"])
    
    
    SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$\\mathbf{' + _pi_s_m_one + '}$', 
                    fs=_fonts['fs'], x_pos=0.7, y_pos=0.71, color='black')
    
    
    axins1 = inset_axes(_ax_sk_meta_l_k,
                        width="25%",  # width: 50% of parent_bbox width
                        height="5%",  # height: 5%
                        loc="upper right", 
                        bbox_to_anchor=[-.06, -.3, 1, 1],
                        bbox_transform=_ax_sk_meta_l_k.transAxes
                       )
    fig.colorbar(cm.ScalarMappable(norm=_gs_col_norm,cmap=_cmap), 
                 cax=axins1, orientation="horizontal", ticks=[])

    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_a_fine), vmax = np.amax(_a_fine))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkgreen","darkorange"])
    
    _every = 1
    _i_a_list = [0, 9, 17, 31]
    ## (GSequence(n_steps=5, num=1, den=1) - 1)
    for index, _i_a in enumerate(_i_a_list):
        _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
        _ax_sk_meta_l_k.plot(_k_fine[::_every], np.abs(_G) * _2b_sk_spin_l_ave[index,:][::_every], 
                 marker='o', linestyle='none', color=_color)
        _ax_sk_meta_l_k.plot(_k_fine_alt, _Sk_fig_2b_hat[index,:], color=_color)
        _ax_sk_meta_l_k.plot(_k_fine_alt, _Sk_fig_2b_k2_hat[index,:], color=_color, linestyle='--')
        

    _ax_sk_meta_l_k.set_yscale('log')
    _ax_sk_meta_l_k.set_xscale('log')
    SetAxTicksFont(_ax_sk_meta_l_k, _fonts['fs'])
    _ax_sk_meta_l_k.set_xlabel('$k\Delta x$', fontsize = _fonts['fs'])    
    SetAxPanelLabel(ax=_ax_sk_meta_l_k, label=_panel_label, fs=_fonts['fs'], pos='ur')
    _ax_sk_meta_l_k.set_xlim([8e-1 * _k_fine[1], np.pi])
    _ax_sk_meta_l_k.set_ylim([1.3e-12, 8e-10])
        
    _ax_sk_meta_l_k.set_xticks([np.pi/32, np.pi/16, np.pi/8, np.pi/4, np.pi/2, np.pi])
    _ax_sk_meta_l_k.set_xticklabels(['$\pi/32$', '$\pi/16$', '$\pi/8$', '$\pi/4$', '$\pi/2$', '$\pi$'])
    
    
    _delta_Ps_liq = (_Pb(_n_spin_liq)) / float(_p_spin_l)    
    
    _i_a = _i_a_list[3]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = _delta_Ps_liq[3] - 1
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.01, y_pos=0.75, color=_color, rotation=-37)

    _i_a = _i_a_list[2]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = _delta_Ps_liq[2] - 1
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.01, y_pos=0.58, color=_color, rotation=-12)

    _i_a = _i_a_list[1]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = _delta_Ps_liq[1] - 1
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.45, color=_color, rotation=-4)

    
    _i_a = _i_a_list[0]
    _color = _cmap(_gs_col_norm(_a_fine[_i_a]))
    _label_value = _delta_Ps_liq[0] - 1
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='$\\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.3, color=_color, rotation=0)        
        
    SetAxPanelLabel(ax=_ax_sk_meta_l_k, label='{Liquid} -- ${\\hat{T} = ' + ("%.1f" % (_G_c/_G)) + '}$', 
                    fs=_fonts['fs'], x_pos=0.4, y_pos=0.9)

#############################################################################

_ax_sk_crit_g_k = plt.subplot2grid((_ny_fig, _nx_fig), (1, 0), colspan = 1, rowspan = 1)
_panel_label="$(c)$"

if True:
    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_delta_Ts_gas), vmax = np.amax(_delta_Ts_gas))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkorange","darkgreen"])
    
    
    SetAxPanelLabel(ax=_ax_sk_crit_g_k, label='$\\mathbf{1 - \\hat{T}}$', 
                    fs=_fonts['fs'], x_pos=0.7, y_pos=0.71, color='black')
    
    
    axins1 = inset_axes(_ax_sk_crit_g_k,
                        width="25%",  # width: 50% of parent_bbox width
                        height="5%",  # height: 5%
                        loc="upper right", 
                        bbox_to_anchor=[-.065, -.3, 1, 1],
                        bbox_transform=_ax_sk_crit_g_k.transAxes
                       )
    fig.colorbar(cm.ScalarMappable(norm=_gs_col_norm,cmap=_cmap), 
                 cax=axins1, orientation="horizontal", ticks=[])    
    
    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_delta_Ts_gas), vmax = np.amax(_delta_Ts_gas))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkorange","darkgreen"])
    
    _g_i_list = [0, 6, 14, 31]
    _every = 2
    for index, _g_i in enumerate(_g_i_list):
        _color = _cmap(_gs_col_norm(_delta_Ts_liq[index]))
        G_value = _G_fine[_g_i]
        _ax_sk_crit_g_k.plot(_k_fine[::_every], np.abs(G_value) * _2c_sk_spin_g_ave[index,:][::_every], 'o', 
                             color=_color, linestyle='none')
        _ax_sk_crit_g_k.plot(_k_fine_alt, _Sk_fig_2c_hat[index,:], color=_color)
        _ax_sk_crit_g_k.plot(_k_fine_alt, _Sk_fig_2c_k2_hat[index,:], color=_color, linestyle='--')

    SetAxPanelLabel(ax=_ax_sk_crit_g_k, label=_panel_label, fs=_fonts['fs'], pos='ur')
    SetAxPanelLabel(ax=_ax_sk_crit_g_k, label='{Gas} -- binodal', 
                    fs=_fonts['fs'], x_pos=0.45, y_pos=0.91)    
    
    _ax_sk_crit_g_k.set_yscale('log')
    _ax_sk_crit_g_k.set_xscale('log')

    _ax_sk_crit_g_k.set_ylim([8e-14, 6e-9])
    
    _ax_sk_crit_g_k.set_xticks([np.pi/64, np.pi/16, np.pi/4, np.pi])
    _ax_sk_crit_g_k.set_xticklabels(['$\pi/64$', '$\pi/16$', '$\pi/4$', '$\pi$'])
    _ax_sk_crit_g_k.set_xlim([4e-1 * _k_fine[0], np.pi])    
    
    SetAxTicksFont(_ax_sk_crit_g_k, _fonts['fs'])
    _ax_sk_crit_g_k.set_xlabel('$k\Delta x$', fontsize = _fonts['fs'])
    _ax_sk_crit_g_k.set_ylabel('$\hat{S}(k\Delta x)$', fontsize = _fonts['fs'])    
    
    index = 3
    _color = _cmap(_gs_col_norm(_delta_Ts_gas[index]))
    _label_value = _delta_Ts_gas[index]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_crit_g_k, label='$\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.005, y_pos=0.71, color=_color, rotation=-33.5)
        
        
    index = 2
    _color = _cmap(_gs_col_norm(_delta_Ts_gas[index]))
    _label_value = _delta_Ts_gas[index]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_crit_g_k, label='$\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.52, color=_color, rotation=-8)
        
    index = 1
    _color = _cmap(_gs_col_norm(_delta_Ts_gas[index]))
    _label_value = _delta_Ts_gas[index]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_crit_g_k, label='$\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.27, color=_color, rotation=0)
        
    index = 0
    _color = _cmap(_gs_col_norm(_delta_Ts_gas[index]))
    _label_value = _delta_Ts_gas[index]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_crit_g_k, label='$\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.05, color=_color, rotation=0)

#############################################################################

_ax_sk_crit_l_k = plt.subplot2grid((_ny_fig, _nx_fig), (1, 1), colspan = 1, rowspan = 1)
_panel_label="$(d)$"

if True:
    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_delta_Ts_liq), vmax = np.amax(_delta_Ts_liq))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkorange","darkgreen"])
    
    
    SetAxPanelLabel(ax=_ax_sk_crit_l_k, label='$\\mathbf{1 - \\hat{T}}$', 
                    fs=_fonts['fs'], x_pos=0.7, y_pos=0.71, color='black')
    
    
    axins1 = inset_axes(_ax_sk_crit_l_k,
                        width="25%",  # width: 50% of parent_bbox width
                        height="5%",  # height: 5%
                        loc="upper right", 
                        bbox_to_anchor=[-.065, -.3, 1, 1],
                        bbox_transform=_ax_sk_crit_l_k.transAxes
                       )
    fig.colorbar(cm.ScalarMappable(norm=_gs_col_norm,cmap=_cmap), 
                 cax=axins1, orientation="horizontal", ticks=[])
    
    _gs_col_norm = matplotlib.colors.Normalize(vmin = np.amin(_delta_Ts_liq), vmax = np.amax(_delta_Ts_liq))
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["darkorange","darkgreen"])
    
    _g_i_list = [0, 9, 14, 31]
    _every = 2
    for index, _g_i in enumerate(_g_i_list):
        _color = _cmap(_gs_col_norm(_delta_Ts_liq[index]))
        G_value = _G_fine[_g_i]
        _ax_sk_crit_l_k.plot(_k_fine[::_every], np.abs(G_value) * _2d_sk_spin_l_ave[index,:][::_every], 'o', 
                             color=_color, linestyle='none')
        _ax_sk_crit_l_k.plot(_k_fine_alt, _Sk_fig_2d_hat[index,:], color=_color)
        _ax_sk_crit_l_k.plot(_k_fine_alt, _Sk_fig_2d_k2_hat[index,:], color=_color, linestyle='--')
        
    SetAxPanelLabel(ax=_ax_sk_crit_l_k, label=_panel_label, fs=_fonts['fs'], pos='ur')
    SetAxPanelLabel(ax=_ax_sk_crit_l_k, label='{Liquid} -- binodal', 
                    fs=_fonts['fs'], x_pos=0.4, y_pos=0.91)            

    _ax_sk_crit_l_k.set_yscale('log')
    _ax_sk_crit_l_k.set_xscale('log')

    _ax_sk_crit_l_k.set_ylim([7e-13, 8e-9])        
    
    _ax_sk_crit_l_k.set_xticks([np.pi/64, np.pi/16, np.pi/4, np.pi])
    _ax_sk_crit_l_k.set_xticklabels(['$\pi/64$', '$\pi/16$', '$\pi/4$', '$\pi$'])
    _ax_sk_crit_l_k.set_xlim([4e-1 * _k_fine[0], np.pi])    
    
    SetAxTicksFont(_ax_sk_crit_l_k, _fonts['fs'])
    _ax_sk_crit_l_k.set_xlabel('$k\Delta x$', fontsize = _fonts['fs'])
    
    index = 3
    _color = _cmap(_gs_col_norm(_delta_Ts_liq[index]))
    _label_value = _delta_Ts_liq[index]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_crit_l_k, label='$\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.005, y_pos=0.65, color=_color, rotation=-40)
        
    index = 2
    _color = _cmap(_gs_col_norm(_delta_Ts_liq[index]))
    _label_value = _delta_Ts_liq[index]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_crit_l_k, label='$\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.025, y_pos=0.43, color=_color, rotation=-10)
    
    index = 1
    _color = _cmap(_gs_col_norm(_delta_Ts_liq[index]))
    _label_value = _delta_Ts_liq[index]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_crit_l_k, label='$\mathbf{' + _label_value_ltx + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.31, color=_color, rotation=0)
        
    index = 0
    _color = _cmap(_gs_col_norm(_delta_Ts_liq[index]))
    _label_value = _delta_Ts_liq[index]
    _label_value_ltx = LatexESpecifier(_label_value, precision=0, times='\\times')
    SetAxPanelLabel(ax=_ax_sk_crit_l_k, label='$\mathbf{' + _label_value_ltx + '}$', 
                fs=_fonts['fs'], x_pos=0.015, y_pos=0.18, color=_color, rotation=0)    

    
from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

plt.savefig(reproduced_figures / ('figure_2.pdf'), dpi = 300)
plt.savefig(reproduced_figures / ('figure_2.png'), bbox_inches = 'tight', dpi = 300)

## Figure 3

### Simulations

In [ ]:
# Simulations/Data Reading for Fig 3(a)

'''
Data for Fig 3(a)
'''
print("Figure 3(a) data")
if True:
    _L, _tau = 128, 1
    figure_3a_a_vals, _G, _kBT = _a_fine[::2], -3, 1e-13

    n = sp.Symbol('n')
    psi_sym = sp.exp(-1/n)
    psi_code = 'exp((NType)(-1./ln_0))'

    from idpy.Utils.SimpleTiming import SimpleTiming
    _st = SimpleTiming()

    for seed, a_value in enumerate(figure_3a_a_vals):

        _s_data = StructureFactorsData(dim=dimension, L=_L, n_type='n_g', a=a_value, G=_G, kBT=_kBT)
        N_check = _s_data.CheckContent()

        print("Checking if the data is already there...", end="")
        if N_check != (_time_tuple[1] // _time_tuple[2]):
            print("no!")
            print("Running the simulation")
            # Declaring simulation object _test_meta
            matastable_sim = \
                ShanChenMultiPhase(dim_sizes = (_L,) * dimension, 
                                   xi_stencil = _xi_stencil, 
                                   f_stencil = _f_stencil,
                                   psi_code = psi_code, 
                                   psi_sym = psi_sym, 
                                   e2_val = 1, 
                                   SC_G = _G, tau = _tau,
                                   lang = preferred_lang, device = preferred_device, 
                                   cl_kind = preferred_kind, 
                                   optimizer_flag = True, 
                                   fluctuations = 'Gross2011', 
                                   prng_distribution = 'gaussian', 
                                   indep_gaussian = False, 
                                   prng_kind = 'MMIX',
                                   prng_init_from = 'numpy', 
                                   init_seed=seed)

            _n_start = GetDensitySpinodalDistance(a_value, psi_sym, _G, 'n_g')

            matastable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                             width = _L//2, direction = 0)


            _st.Start()
            matastable_sim.MainLoopGross2011SRT(
                time_steps = range(*_time_tuple), 
                convergence_functions = [_structure_factor_analysis],
                profiling = False,
                kBT = _kBT, n0 = _n_start
            )
            _st.End()

            _st.PrintElapsedTime()

            _n2_ft_data = np.array(matastable_sim.sims_vars['n2_ft'])

            _n2_ft_ave, _n2_ft_err = [], []
            for _i in range(_n2_ft_data.shape[1]):
                _mean_swap, _err_swap = ComputeMeanErr(_n2_ft_data[:,_i])    
                _n2_ft_ave += [_mean_swap]
                _n2_ft_err += [_err_swap]    

            _n2_ft_ave, _n2_ft_err = np.array(_n2_ft_ave), np.array(_n2_ft_err)

            _k_range = matastable_sim.sims_vars['nx_list'] * 2 * np.pi / _L    

            _data_dict = \
                {'k_range': _k_range[1:], 'n_start': _n_start, 
                 'Sk': {'ave': _n2_ft_ave[1:], 'err': _n2_ft_err[1:], 
                        'N': len(_n2_ft_data[:,0])}}


            _s_data.PushData(data_dict=_data_dict)

            print("-------------------------------------------------------------------------")
            print()

            matastable_sim.End()
        else:
            print("yes!")

In [ ]:
# Simulations/Data Reading for Fig 3(b)

'''
Data for Fig 3(b)
'''
print("Figure 3(b) data")
if True:
    _L, _tau = 128, 1
    figure_3b_a_vals, _G, _kBT = _a_fine[::2], -3, 1e-13

    n = sp.Symbol('n')
    psi_sym = sp.exp(-1/n)
    psi_code = 'exp((NType)(-1./ln_0))'

    from idpy.Utils.SimpleTiming import SimpleTiming
    _st = SimpleTiming()

    for seed, a_value in enumerate(figure_3b_a_vals):

        _s_data = StructureFactorsData(dim=dimension, L=_L, n_type='n_l', a=a_value, G=_G, kBT=_kBT)
        N_check = _s_data.CheckContent()

        print("Checking if the data is already there...", end="")
        if N_check != (_time_tuple[1] // _time_tuple[2]):
            print("no!")
            print("Running the simulation")
            # Declaring simulation object _test_meta
            matastable_sim = \
                ShanChenMultiPhase(dim_sizes = (_L,) * dimension, 
                                   xi_stencil = _xi_stencil, 
                                   f_stencil = _f_stencil,
                                   psi_code = psi_code, 
                                   psi_sym = psi_sym, 
                                   e2_val = 1, 
                                   SC_G = _G, tau = _tau,
                                   lang = preferred_lang, device = preferred_device, 
                                   cl_kind = preferred_kind, 
                                   optimizer_flag = True, 
                                   fluctuations = 'Gross2011', 
                                   prng_distribution = 'gaussian', 
                                   indep_gaussian = False, 
                                   prng_kind = 'MMIX',
                                   prng_init_from = 'numpy', 
                                   init_seed=seed)

            _n_start = GetDensitySpinodalDistance(a_value, psi_sym, _G, 'n_l')

            matastable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                             width = _L//2, direction = 0)


            _st.Start()
            matastable_sim.MainLoopGross2011SRT(
                time_steps = range(*_time_tuple), 
                convergence_functions = [_structure_factor_analysis],
                profiling = False,
                kBT = _kBT, n0 = _n_start
            )
            _st.End()

            _st.PrintElapsedTime()

            _n2_ft_data = np.array(matastable_sim.sims_vars['n2_ft'])

            _n2_ft_ave, _n2_ft_err = [], []
            for _i in range(_n2_ft_data.shape[1]):
                _mean_swap, _err_swap = ComputeMeanErr(_n2_ft_data[:,_i])    
                _n2_ft_ave += [_mean_swap]
                _n2_ft_err += [_err_swap]    

            _n2_ft_ave, _n2_ft_err = np.array(_n2_ft_ave), np.array(_n2_ft_err)

            _k_range = matastable_sim.sims_vars['nx_list'] * 2 * np.pi / _L    

            _data_dict = \
                {'k_range': _k_range[1:], 'n_start': _n_start, 
                 'Sk': {'ave': _n2_ft_ave[1:], 'err': _n2_ft_err[1:], 
                        'N': len(_n2_ft_data[:,0])}}


            _s_data.PushData(data_dict=_data_dict)

            print("-------------------------------------------------------------------------")
            print()

            matastable_sim.End()
        else:
            print("yes!")

In [ ]:
# Simulations/Data Reading for Fig 3(c)
'''
Data for Fig 3(c)
'''
print("Figure 3(c) data")
if True:
    _L, _tau = 512, 1
    figure_3c_G_list = _G_fine[::2]

    n = sp.Symbol('n')
    psi_sym = sp.exp(-1/n)
    psi_code = 'exp((NType)(-1./ln_0))'

    from idpy.Utils.SimpleTiming import SimpleTiming
    _st = SimpleTiming()
    
    for seed, G_value in enumerate(figure_3c_G_list):

        _s_data = StructureFactorsData(dim=dimension, L=_L, n_type='n_g', a=0, G=G_value, kBT=_kBT)
        N_check = _s_data.CheckContent()

        print("Checking if the data is already there...", end="")        
        if N_check != (_time_tuple[1] // _time_tuple[2]):
            print("no!")
            print("Running the simulation")            
            # Declaring simulation object _test_meta
            matastable_sim = \
                ShanChenMultiPhase(dim_sizes = (_L,) * dimension, 
                                   xi_stencil = _xi_stencil, 
                                   f_stencil = _f_stencil,
                                   psi_code = psi_code, 
                                   psi_sym = psi_sym, 
                                   e2_val = 1, 
                                   SC_G = G_value, tau = _tau,
                                   lang = preferred_lang, device = preferred_device, 
                                   cl_kind = preferred_kind, 
                                   optimizer_flag = True, 
                                   fluctuations = 'Gross2011', 
                                   prng_distribution = 'gaussian', 
                                   indep_gaussian = False, 
                                   prng_kind = 'MMIX',
                                   prng_init_from = 'numpy', 
                                   init_seed=seed)

            _n_start = GetDensitySpinodalDistance(0, psi_sym, G_value, 'n_g')

            matastable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                             width = _L//2, direction = 0)

            _st.Start()
            matastable_sim.MainLoopGross2011SRT(
                time_steps = range(*_time_tuple), 
                convergence_functions = [_structure_factor_analysis],
                profiling = False,
                kBT = _kBT, n0 = _n_start
            )
            _st.End()

            _st.PrintElapsedTime()

            _n2_ft_data = np.array(matastable_sim.sims_vars['n2_ft'])

            _n2_ft_ave, _n2_ft_err = [], []
            for _i in range(_n2_ft_data.shape[1]):
                _mean_swap, _err_swap = ComputeMeanErr(_n2_ft_data[:,_i])    
                _n2_ft_ave += [_mean_swap]
                _n2_ft_err += [_err_swap]    

            _n2_ft_ave, _n2_ft_err = np.array(_n2_ft_ave), np.array(_n2_ft_err)

            _k_range = matastable_sim.sims_vars['nx_list'] * 2 * np.pi / _L    

            _data_dict = \
                {'k_range': _k_range[1:], 'n_start': _n_start, 
                 'Sk': {'ave': _n2_ft_ave[1:], 'err': _n2_ft_err[1:], 
                        'N': len(_n2_ft_data[:,0])}}


            _s_data.PushData(data_dict=_data_dict)

            print("-------------------------------------------------------------------------")
            print()

            matastable_sim.End()
        else:
            print("yes!")            

In [ ]:
# Simulations/Data Reading for Fig 3(d)
'''
Data for Fig 3(d)
'''
print("Figure 3(d) data")
if True:
    _L, _tau = 512, 1
    figure_3d_G_list = _G_fine[::2]

    n = sp.Symbol('n')
    psi_sym = sp.exp(-1/n)
    psi_code = 'exp((NType)(-1./ln_0))'

    from idpy.Utils.SimpleTiming import SimpleTiming
    _st = SimpleTiming()
    
    for seed, G_value in enumerate(figure_3d_G_list):

        _s_data = StructureFactorsData(dim=dimension, L=_L, n_type='n_l', a=0, G=G_value, kBT=_kBT)
        N_check = _s_data.CheckContent()

        print("Checking if the data is already there...", end="")        
        if N_check != (_time_tuple[1] // _time_tuple[2]):
            print("no!")
            print("Running the simulation")            
            # Declaring simulation object _test_meta
            matastable_sim = \
                ShanChenMultiPhase(dim_sizes = (_L,) * dimension, 
                                   xi_stencil = _xi_stencil, 
                                   f_stencil = _f_stencil,
                                   psi_code = psi_code, 
                                   psi_sym = psi_sym, 
                                   e2_val = 1, 
                                   SC_G = G_value, tau = _tau,
                                   lang = preferred_lang, device = preferred_device, 
                                   cl_kind = preferred_kind, 
                                   optimizer_flag = True, 
                                   fluctuations = 'Gross2011', 
                                   prng_distribution = 'gaussian', 
                                   indep_gaussian = False, 
                                   prng_kind = 'MMIX',
                                   prng_init_from = 'numpy', 
                                   init_seed=seed)

            _n_start = GetDensitySpinodalDistance(0, psi_sym, G_value, 'n_l')

            matastable_sim.InitFlatInterface(n_g = _n_start, n_l = _n_start, 
                                             width = _L//2, direction = 0)

            _st.Start()
            matastable_sim.MainLoopGross2011SRT(
                time_steps = range(*_time_tuple), 
                convergence_functions = [_structure_factor_analysis],
                profiling = False,
                kBT = _kBT, n0 = _n_start
            )
            _st.End()

            _st.PrintElapsedTime()

            _n2_ft_data = np.array(matastable_sim.sims_vars['n2_ft'])

            _n2_ft_ave, _n2_ft_err = [], []
            for _i in range(_n2_ft_data.shape[1]):
                _mean_swap, _err_swap = ComputeMeanErr(_n2_ft_data[:,_i])    
                _n2_ft_ave += [_mean_swap]
                _n2_ft_err += [_err_swap]    

            _n2_ft_ave, _n2_ft_err = np.array(_n2_ft_ave), np.array(_n2_ft_err)

            _k_range = matastable_sim.sims_vars['nx_list'] * 2 * np.pi / _L    

            _data_dict = \
                {'k_range': _k_range[1:], 'n_start': _n_start, 
                 'Sk': {'ave': _n2_ft_ave[1:], 'err': _n2_ft_err[1:], 
                        'N': len(_n2_ft_data[:,0])}}


            _s_data.PushData(data_dict=_data_dict)

            print("-------------------------------------------------------------------------")
            print()

            matastable_sim.End()
        else:
            print("yes!")            

In [ ]:
# Collecting data from the json file
'''
Values for the relative distance from the 
'''
_a_fine = 1 - np.exp(np.linspace(np.log(1e-2), np.log(1), 2 ** 5))
_a_fine = np.flip(_a_fine)

_a_fine_G = 1 - np.exp(np.linspace(np.log(1e-5), np.log(1), 2 ** 5))
_a_fine_G = np.flip(_a_fine_G)
_a_fine_G

_G_c = -np.exp(2) / 3
'''
The factor 1.56 has been manually set for tuning the value of G
near the critical temperature
'''
_G_fine = _a_fine_G * _G_c + (1 - _a_fine_G) * (_G_c * (1.56))


'''
Fig 3(a)
'''
_L, _tau = 128, 1
figure_3a_a_vals, _G, _kBT = _a_fine[::2], -3, 1e-13

_data_spin_ng_3a = []

for a_value in figure_3a_a_vals:
    _sf_data = \
        StructureFactorsData(dim=dimension, L=_L, n_type='n_g', a=a_value, G=_G, kBT=_kBT)
    
    _data_spin_ng_3a += [_sf_data.PullData(_sf_data.DataKeyPrefix())]
    
_data_spin_ng_3a = np.array(_data_spin_ng_3a)

'''
Fig 3(b)
'''
_L, _tau = 128, 1
figure_3b_a_vals, _G, _kBT = _a_fine[::2], -3, 1e-13

_data_spin_nl_3b = []

for a_value in figure_3b_a_vals:
    _sf_data = \
        StructureFactorsData(dim=dimension, L=_L, n_type='n_l', a=a_value, G=_G, kBT=_kBT)
    
    _data_spin_nl_3b += [_sf_data.PullData(_sf_data.DataKeyPrefix())]
    
_data_spin_nl_3b = np.array(_data_spin_nl_3b)

'''
Fig 3(c)
'''
_L, _tau = 512, 1
figure_3c_G_list, _G, _kBT = _G_fine[::2], -3, 1e-13

_data_bin_ng_3c = []

for G_value in figure_3c_G_list:
    _sf_data = \
        StructureFactorsData(dim=dimension, L=_L, n_type='n_g', a=0, G=G_value, kBT=_kBT)
    
    _data_bin_ng_3c += [_sf_data.PullData(_sf_data.DataKeyPrefix())]
    
_data_bin_ng_3c = np.array(_data_bin_ng_3c)

'''
Fig 3(d)
'''
_L, _tau = 512, 1
figure_3d_G_list, _G, _kBT = _G_fine[::2], -3, 1e-13

_data_bin_nl_3d = []

for G_value in figure_3d_G_list:
    _sf_data = \
        StructureFactorsData(dim=dimension, L=_L, n_type='n_l', a=0, G=G_value, kBT=_kBT)
    
    _data_bin_nl_3d += [_sf_data.PullData(_sf_data.DataKeyPrefix())]
    
_data_bin_nl_3d = np.array(_data_bin_nl_3d)

In [ ]:
# Some more data handling...
'''
I need to collect the data for each 'k' as a function of 'a'
The simplest way to do it is to create a higher order numpy array
'''
_kBT = 1e-13

'''
Metastable gas
'''
_all_sk_spin_g_ave_3a, _all_sk_spin_g_err_3a, _n_spin_gs_3a = [], [], []
for _i in range(len(_data_spin_ng_3a)):
    _n_spin_gs_3a += [_data_spin_ng_3a[_i]['n_start']]
    _all_sk_spin_g_ave_3a += [np.array(_data_spin_ng_3a[_i]['Sk']['ave'])]
    _all_sk_spin_g_err_3a += [np.array(_data_spin_ng_3a[_i]['Sk']['err'])]

_all_sk_spin_g_ave_3a = np.array(_all_sk_spin_g_ave_3a)
_all_sk_spin_g_err_3a = np.array(_all_sk_spin_g_err_3a)
_n_spin_gs_3a = np.array(_n_spin_gs_3a)

'''
Metastable liquid
'''
_all_sk_spin_l_ave_3b, _all_sk_spin_l_err_3b, _n_spin_gs_3b = [], [], []
for _i in range(len(_data_spin_nl_3b)):
    _n_spin_gs_3b += [_data_spin_nl_3b[_i]['n_start']]
    _all_sk_spin_l_ave_3b += [np.array(_data_spin_nl_3b[_i]['Sk']['ave'])]
    _all_sk_spin_l_err_3b += [np.array(_data_spin_nl_3b[_i]['Sk']['err'])]

_all_sk_spin_l_ave_3b = np.array(_all_sk_spin_l_ave_3b)
_all_sk_spin_l_err_3b = np.array(_all_sk_spin_l_err_3b)
_n_spin_gs_3b = np.array(_n_spin_gs_3b)

'''
Binodal gas
'''
_all_sk_bin_g_ave_3c, _all_sk_bin_g_err_3c, _n_bin_gs_3c = [], [], []
for _i in range(len(_data_bin_ng_3c)):
    _n_bin_gs_3c += [_data_bin_ng_3c[_i]['n_start']]
    _all_sk_bin_g_ave_3c += [np.array(_data_bin_ng_3c[_i]['Sk']['ave'])]
    _all_sk_bin_g_err_3c += [np.array(_data_bin_ng_3c[_i]['Sk']['err'])]

_all_sk_bin_g_ave_3c = np.array(_all_sk_bin_g_ave_3c)
_all_sk_bin_g_err_3c = np.array(_all_sk_bin_g_err_3c)
_n_bin_gs_3c = np.array(_n_bin_gs_3c)

'''
Binodal liquid
'''
_all_sk_bin_l_ave_3d, _all_sk_bin_l_err_3d, _n_bin_ls_3d = [], [], []
for _i in range(len(_data_bin_nl_3d)):
    _n_bin_ls_3d += [_data_bin_nl_3d[_i]['n_start']]
    _all_sk_bin_l_ave_3d += [np.array(_data_bin_nl_3d[_i]['Sk']['ave'])]
    _all_sk_bin_l_err_3d += [np.array(_data_bin_nl_3d[_i]['Sk']['err'])]

_all_sk_bin_l_ave_3d = np.array(_all_sk_bin_l_ave_3d)
_all_sk_bin_l_err_3d = np.array(_all_sk_bin_l_err_3d)
_n_bin_ls_3d = np.array(_n_bin_ls_3d)


In [ ]:
# Generating the curves according to the analytical results

_k_fine_alt = np.linspace(0, np.pi, 2 ** 8)
_k_fine_alt_512 = np.linspace(0, np.pi, 2 ** 9)
_k_fine = np.array(_data_spin_ng_3a[0]['k_range'])
_k_fine_512 = np.array(_data_bin_ng_3c[0]['k_range'])

def Sk2(k, c_s2, Gcs2, n0, kBT, psi_sym):
    _psi_f = sp.lambdify(n, psi_sym)
    _d_psi_f = sp.lambdify(n, psi_sym.diff())
    _psi0 = _psi_f(n0)
    _dpsi0 = _d_psi_f(n0)
    _val = \
        n0 * kBT / \
        (c_s2 + Gcs2 * (_psi0 * _dpsi0) - (k ** 2) * Gcs2 * _psi0 * _dpsi0 / 4)
    return _val

def Sk(k, c_s2, Gcs2, n0, kBT, psi_sym):
    _psi_f = sp.lambdify(n, psi_sym)
    _d_psi_f = sp.lambdify(n, psi_sym.diff())
    _psi0 = _psi_f(n0)
    _dpsi0 = _d_psi_f(n0)
    _val = \
        n0 * kBT / \
        (c_s2 + Gcs2 * (_psi0 * _dpsi0) + Gcs2 * _psi0 * _dpsi0 * (np.cos(k) - 1) / 2)
    return _val

def Sk2Hat(k, T_hat, n0_hat, Theta_hat, psi_sym):
    _psi_f = sp.lambdify(n, psi_sym)
    _d_psi_f = sp.lambdify(n, psi_sym.diff())
    _psi0 = _psi_f(n0_hat)
    _dpsi0 = _d_psi_f(n0_hat)
    _val = \
        2. * n0_hat * Theta_hat / \
        (2. * T_hat - 2. * (_psi0 * _dpsi0) + (k ** 2) * _psi0 * _dpsi0 / 2)
    return _val

def SkHat(k, T_hat, n0_hat, Theta_hat, psi_sym):
    _psi_f = sp.lambdify(n, psi_sym)
    _d_psi_f = sp.lambdify(n, psi_sym.diff())
    _psi0 = _psi_f(n0_hat)
    _dpsi0 = _d_psi_f(n0_hat)
    _val = \
        2. * n0_hat * Theta_hat / \
        (2. * T_hat - 2. * (_psi0 * _dpsi0) - _psi0 * _dpsi0 * (np.cos(k) - 1))
    return _val


psi_sym = sp.exp(-1/n)
psi_sym_hat = sp.exp(1-1/n)

'''
Fig 3(a) theoretical values
'''
_Sk_fig_3a, _Sk_fig_3a_k2 = [], []
_Sk_fig_3a_hat, _Sk_fig_3a_k2_hat = [], []
_Sk_fig_3a_data_match, _Sk_fig_3a_k2_data_match = [], []
_Sk_fig_3a_hat_data_match, _Sk_fig_3a_k2_hat_data_match = [], []


_n_spin_gs = []

for a_value in figure_3a_a_vals:
    _n_init = GetDensitySpinodalDistance(a_value, psi_sym, _G, d_type='n_g')
    _sk_lambda = lambda k: Sk(k, 1/3, _G, _n_init, _kBT, psi_sym)
    _sk_lambda_k2 = lambda k: Sk2(k, 1/3, _G, _n_init, _kBT, psi_sym)

    T_hat = np.exp(2) / np.abs(_G * 3)
    Theta_hat = _kBT * np.exp(2)
    _sk_lambda_hat = lambda k: SkHat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    _sk_lambda_k2_hat = lambda k: Sk2Hat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    
    print("Gas density:", _n_init)

    _Sk_fig_3a += [[_sk_lambda(_k) for _k in _k_fine_alt]]
    _Sk_fig_3a_k2 += [[_sk_lambda_k2(_k) for _k in _k_fine_alt]]
    _Sk_fig_3a_hat += [[_sk_lambda_hat(_k) for _k in _k_fine_alt]]
    _Sk_fig_3a_k2_hat += [[_sk_lambda_k2_hat(_k) for _k in _k_fine_alt]]

    _Sk_fig_3a_data_match += [[_sk_lambda(_k) for _k in _k_fine]]
    _Sk_fig_3a_k2_data_match += [[_sk_lambda_k2(_k) for _k in _k_fine]]
    _Sk_fig_3a_hat_data_match += [[_sk_lambda_hat(_k) for _k in _k_fine]]
    _Sk_fig_3a_k2_hat_data_match += [[_sk_lambda_k2_hat(_k) for _k in _k_fine]]

_Sk_fig_3a, _Sk_fig_3a_k2 = \
    np.array(_Sk_fig_3a), np.array(_Sk_fig_3a_k2)
_Sk_fig_3a_hat, _Sk_fig_3a_k2_hat = \
    np.array(_Sk_fig_3a_hat), np.array(_Sk_fig_3a_k2_hat)

_Sk_fig_3a_data_match, _Sk_fig_3a_k2_data_match = \
    np.array(_Sk_fig_3a_data_match), np.array(_Sk_fig_3a_k2_data_match)
_Sk_fig_3a_hat_data_match, _Sk_fig_3a_k2_hat_data_match = \
    np.array(_Sk_fig_3a_hat_data_match), np.array(_Sk_fig_3a_k2_hat_data_match)

'''
Fig 3(b) theoretical values
'''
_Sk_fig_3b, _Sk_fig_3b_k2 = [], []
_Sk_fig_3b_hat, _Sk_fig_3b_k2_hat = [], []
_Sk_fig_3b_data_match, _Sk_fig_3b_k2_data_match = [], []
_Sk_fig_3b_hat_data_match, _Sk_fig_3b_k2_hat_data_match = [], []


_n_spin_ls = []

for a_value in figure_3b_a_vals:
    _n_init = GetDensitySpinodalDistance(a_value, psi_sym, _G, d_type='n_l')
    _sk_lambda = lambda k: Sk(k, 1/3, _G, _n_init, _kBT, psi_sym)
    _sk_lambda_k2 = lambda k: Sk2(k, 1/3, _G, _n_init, _kBT, psi_sym)

    T_hat = np.exp(2) / np.abs(_G * 3)
    Theta_hat = _kBT * np.exp(2)
    _sk_lambda_hat = lambda k: SkHat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    _sk_lambda_k2_hat = lambda k: Sk2Hat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    
    print("Liquid density:", _n_init)

    _Sk_fig_3b += [[_sk_lambda(_k) for _k in _k_fine_alt]]
    _Sk_fig_3b_k2 += [[_sk_lambda_k2(_k) for _k in _k_fine_alt]]
    _Sk_fig_3b_hat += [[_sk_lambda_hat(_k) for _k in _k_fine_alt]]
    _Sk_fig_3b_k2_hat += [[_sk_lambda_k2_hat(_k) for _k in _k_fine_alt]]

    _Sk_fig_3b_data_match += [[_sk_lambda(_k) for _k in _k_fine]]
    _Sk_fig_3b_k2_data_match += [[_sk_lambda_k2(_k) for _k in _k_fine]]
    _Sk_fig_3b_hat_data_match += [[_sk_lambda_hat(_k) for _k in _k_fine]]
    _Sk_fig_3b_k2_hat_data_match += [[_sk_lambda_k2_hat(_k) for _k in _k_fine]]

_Sk_fig_3b, _Sk_fig_3b_k2 = \
    np.array(_Sk_fig_3b), np.array(_Sk_fig_3b_k2)
_Sk_fig_3b_hat, _Sk_fig_3b_k2_hat = \
    np.array(_Sk_fig_3b_hat), np.array(_Sk_fig_3b_k2_hat)

_Sk_fig_3b_data_match, _Sk_fig_3b_k2_data_match = \
    np.array(_Sk_fig_3b_data_match), np.array(_Sk_fig_3b_k2_data_match)
_Sk_fig_3b_hat_data_match, _Sk_fig_3b_k2_hat_data_match = \
    np.array(_Sk_fig_3b_hat_data_match), np.array(_Sk_fig_3b_k2_hat_data_match)

'''
Fig 3(c) theoretical values
'''
_Sk_fig_3c, _Sk_fig_3c_k2 = [], []
_Sk_fig_3c_hat, _Sk_fig_3c_k2_hat = [], []
_Sk_fig_3c_data_match, _Sk_fig_3c_k2_data_match = [], []
_Sk_fig_3c_hat_data_match, _Sk_fig_3c_k2_hat_data_match = [], []


_n_bin_gs = []

for G_value in figure_3c_G_list:
    _n_init = GetDensitySpinodalDistance(0, psi_sym, G_value, d_type='n_g')
    _sk_lambda = lambda k: Sk(k, 1/3, G_value, _n_init, _kBT, psi_sym)
    _sk_lambda_k2 = lambda k: Sk2(k, 1/3, G_value, _n_init, _kBT, psi_sym)

    T_hat = np.exp(2) / np.abs(G_value * 3)
    Theta_hat = _kBT * np.exp(2)
    _sk_lambda_hat = lambda k: SkHat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    _sk_lambda_k2_hat = lambda k: Sk2Hat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    
    print("Gas density:", _n_init)

    _Sk_fig_3c += [[_sk_lambda(_k) for _k in _k_fine_alt_512]]
    _Sk_fig_3c_k2 += [[_sk_lambda_k2(_k) for _k in _k_fine_alt_512]]
    _Sk_fig_3c_hat += [[_sk_lambda_hat(_k) for _k in _k_fine_alt_512]]
    _Sk_fig_3c_k2_hat += [[_sk_lambda_k2_hat(_k) for _k in _k_fine_alt_512]]

    _Sk_fig_3c_data_match += [[_sk_lambda(_k) for _k in _k_fine_512]]
    _Sk_fig_3c_k2_data_match += [[_sk_lambda_k2(_k) for _k in _k_fine_512]]
    _Sk_fig_3c_hat_data_match += [[_sk_lambda_hat(_k) for _k in _k_fine_512]]
    _Sk_fig_3c_k2_hat_data_match += [[_sk_lambda_k2_hat(_k) for _k in _k_fine_512]]

_Sk_fig_3c, _Sk_fig_3c_k2 = \
    np.array(_Sk_fig_3c), np.array(_Sk_fig_3c_k2)
_Sk_fig_3c_hat, _Sk_fig_3c_k2_hat = \
    np.array(_Sk_fig_3c_hat), np.array(_Sk_fig_3c_k2_hat)

_Sk_fig_3c_data_match, _Sk_fig_3c_k2_data_match = \
    np.array(_Sk_fig_3c_data_match), np.array(_Sk_fig_3c_k2_data_match)
_Sk_fig_3c_hat_data_match, _Sk_fig_3c_k2_hat_data_match = \
    np.array(_Sk_fig_3c_hat_data_match), np.array(_Sk_fig_3c_k2_hat_data_match)

'''
Fig 3(d) theoretical values
'''
_Sk_fig_3d, _Sk_fig_3d_k2 = [], []
_Sk_fig_3d_hat, _Sk_fig_3d_k2_hat = [], []
_Sk_fig_3d_data_match, _Sk_fig_3d_k2_data_match = [], []
_Sk_fig_3d_hat_data_match, _Sk_fig_3d_k2_hat_data_match = [], []


_n_bin_ls = []

for G_value in figure_3d_G_list:
    _n_init = GetDensitySpinodalDistance(0, psi_sym, G_value, d_type='n_l')
    _sk_lambda = lambda k: Sk(k, 1/3, G_value, _n_init, _kBT, psi_sym)
    _sk_lambda_k2 = lambda k: Sk2(k, 1/3, G_value, _n_init, _kBT, psi_sym)

    T_hat = np.exp(2) / np.abs(G_value * 3)
    Theta_hat = _kBT * np.exp(2)
    _sk_lambda_hat = lambda k: SkHat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    _sk_lambda_k2_hat = lambda k: Sk2Hat(k, T_hat, _n_init, Theta_hat, psi_sym_hat)
    
    print("Liquid density:", _n_init)

    _Sk_fig_3d += [[_sk_lambda(_k) for _k in _k_fine_alt_512]]
    _Sk_fig_3d_k2 += [[_sk_lambda_k2(_k) for _k in _k_fine_alt_512]]
    _Sk_fig_3d_hat += [[_sk_lambda_hat(_k) for _k in _k_fine_alt_512]]
    _Sk_fig_3d_k2_hat += [[_sk_lambda_k2_hat(_k) for _k in _k_fine_alt_512]]

    _Sk_fig_3d_data_match += [[_sk_lambda(_k) for _k in _k_fine_512]]
    _Sk_fig_3d_k2_data_match += [[_sk_lambda_k2(_k) for _k in _k_fine_512]]
    _Sk_fig_3d_hat_data_match += [[_sk_lambda_hat(_k) for _k in _k_fine_512]]
    _Sk_fig_3d_k2_hat_data_match += [[_sk_lambda_k2_hat(_k) for _k in _k_fine_512]]

_Sk_fig_3d, _Sk_fig_3d_k2 = \
    np.array(_Sk_fig_3d), np.array(_Sk_fig_3d_k2)
_Sk_fig_3d_hat, _Sk_fig_3d_k2_hat = \
    np.array(_Sk_fig_3d_hat), np.array(_Sk_fig_3d_k2_hat)

_Sk_fig_3d_data_match, _Sk_fig_3d_k2_data_match = \
    np.array(_Sk_fig_3d_data_match), np.array(_Sk_fig_3d_k2_data_match)
_Sk_fig_3d_hat_data_match, _Sk_fig_3d_k2_hat_data_match = \
    np.array(_Sk_fig_3d_hat_data_match), np.array(_Sk_fig_3d_k2_hat_data_match)

In [ ]:
# Plotting Figure 3

from idpy.Utils.Sequences import GSequence

'''
I can normalize the pressure to the spinodal value so that I do not need to use the hat
'''
_alpha_crit = 0.01

_nx_fig, _ny_fig = 2, 2

_ratio = 1.05
fig = CreateFiguresPanels(_nx=_nx_fig,_ny=_ny_fig, _x_size=6.75/_ratio, _y_size=5.8/_ratio)

_k_selection = (GSequence(n_steps=5, num=1, den=1) - 1)[:-1]
_k_selection = [1, 7, 17, 62]
_ax_sk_meta_g = plt.subplot2grid((_ny_fig, _nx_fig), (0, 0), colspan = 1, rowspan = 1)

_panel_label = '$(a)$'

if True:
    _gs_col_norm = matplotlib.colors.Normalize(vmin = 0, vmax = np.pi)
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["r","b"])    
    
    _every = 2
    _delta_Ps_gas = (float(_p_spin_g) - _Pb(_n_spin_gs_3a)) / float(_p_spin_g)

    _ax_sk_meta_g.set_xscale('log')
    _ax_sk_meta_g.set_yscale('log')

    ##_ax_sk_meta_g.set_ylim([7e-14,9e-11])
    _ax_sk_meta_g.set_xlim([_delta_Ps_gas[-1], _delta_Ps_gas[0]])

    SetAxTicksFont(_ax_sk_meta_g, _fonts['fs'])
    _ax_sk_meta_g.set_xlabel('$' + _one_m_pi_s + '$', fontsize = _fonts['fs'])
    _ax_sk_meta_g.set_ylabel('$\hat{S}(k\Delta x)$', fontsize = _fonts['fs'])   
    SetAxPanelLabel(ax=_ax_sk_meta_g, label=_panel_label, fs=_fonts['fs'], pos='ur')
    
    SetAxPanelLabel(ax=_ax_sk_meta_g, label='$\\mathbf{\hat{S}(0) \sim (' + _one_m_pi_s + ')^{-1/2}}$', 
                    fs=_fonts['fs'], x_pos=0.35, y_pos=0.77, color='black')
    
    '''
    Largest alpha for first and last lines: k=0, k=\pi
    '''
    for _i in [0, _Sk_fig_3a.shape[1] - 1]:
        _linestyle = '-.' if _i == 0 else '-'
        _linewidth = 3 if _i == 0 else 1        
        _color = _cmap(_gs_col_norm(_k_fine_alt[_i]))
        _ax_sk_meta_g.plot(_delta_Ps_gas, _Sk_fig_3a_hat[:,_i], color = _color, 
                           linestyle=_linestyle, linewidth=_linewidth)

    '''
    Smaller alpha for the lines in between in order to create the shading
    '''
    for _i in range(1, _Sk_fig_3a_hat.shape[1] - 1):
        _color = _cmap(_gs_col_norm(_k_fine_alt[_i]))
        _ax_sk_meta_g.plot(_delta_Ps_gas, _Sk_fig_3a_hat[:,_i], color = _color, alpha = _alpha_a)

    '''
    Superposition with data
    '''
    for _i in range(len(_k_selection)):
        _color = _cmap(_gs_col_norm(_k_fine[_k_selection[_i]]))
        _ax_sk_meta_g.errorbar(_delta_Ps_gas, 
                               abs(_G) * _all_sk_spin_g_ave_3a[:,_k_selection[_i]], 
                               yerr=abs(_G) * _all_sk_spin_g_err_3a[:,_k_selection[_i]], 
                               linestyle='none', marker='o', color=_color)

        _ax_sk_meta_g.plot(_delta_Ps_gas, _Sk_fig_3a_hat_data_match[:,_k_selection[_i]], color=_color)
        _ax_sk_meta_g.plot(_delta_Ps_gas, _Sk_fig_3a_k2_hat_data_match[:,_k_selection[_i]], color=_color, linestyle='--')        
    
    _ax_sk_meta_g.plot(_delta_Ps_gas, 5.8e-13 * (_delta_Ps_gas ** -0.5), linestyle = '--', 
                       color = 'black', linewidth = 3)

    '''
    k-Labels for the data
    '''
    index = 0
    _color = _cmap(_gs_col_norm(_k_fine[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_meta_g, label='$\mathbf{k \Delta x=' + ('%.2f' % _k_fine[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.6, color=_color, rotation=-21)

    index = 1
    _color = _cmap(_gs_col_norm(_k_fine[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_meta_g, label='$\mathbf{k\Delta x=' + ('%.2f' % _k_fine[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.47, color=_color, rotation=-6)

    index = 2
    _color = _cmap(_gs_col_norm(_k_fine[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_meta_g, label='$\mathbf{k\Delta x=' + ('%.2f' % _k_fine[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.31, color=_color)

    index = 3
    _color = _cmap(_gs_col_norm(_k_fine[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_meta_g, label='$\mathbf{k\Delta x\simeq \pi}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.09, color=_color)
        

######################################################

_ax_sk_meta_l = plt.subplot2grid((_ny_fig, _nx_fig), (0, 1), colspan = 1, rowspan = 1)
_panel_label = "$(b)$"

if True:
    _gs_col_norm = matplotlib.colors.Normalize(vmin = 0, vmax = np.pi)
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["r","b"])
    
    _every = 2
    _delta_Ps_liq = (_Pb(_n_spin_gs_3b) - float(_p_spin_l)) / float(_p_spin_l)

    _ax_sk_meta_l.set_xscale('log')
    _ax_sk_meta_l.set_yscale('log')

    ##_ax_sk_meta_l.set_ylim([1.5e-13, 2e-10])    
    
    SetAxTicksFont(_ax_sk_meta_l, _fonts['fs'])
    _ax_sk_meta_l.set_xlabel('$' + _pi_s_m_one + '$', fontsize = _fonts['fs'])
    _ax_sk_meta_l.set_xlim([_delta_Ps_liq[-1], _delta_Ps_liq[0]])
    SetAxPanelLabel(ax=_ax_sk_meta_l, label=_panel_label, fs=_fonts['fs'], pos='ur')

    '''
    Largest alpha for first and last lines: k=0, k=\pi
    '''
    for _i in [0, _Sk_fig_3b.shape[1] - 1]:
        _linestyle = '-.' if _i == 0 else '-'
        _linewidth = 3 if _i == 0 else 1                
        _color = _cmap(_gs_col_norm(_k_fine_alt[_i]))
        _ax_sk_meta_l.plot(_delta_Ps_liq, _Sk_fig_3b_hat[:,_i], color=_color, 
                           linestyle=_linestyle, linewidth=_linewidth)


    '''
    Smaller alpha for the lines in between in order to create the shading
    '''
    for _i in range(1, _Sk_fig_3b_hat.shape[1] - 1):
        _color = _cmap(_gs_col_norm(_k_fine_alt[_i]))
        _ax_sk_meta_l.plot(_delta_Ps_liq, _Sk_fig_3b_hat[:,_i], color = _color, alpha = _alpha_a)        

    '''
    Superposition with data
    '''        
    for _i in range(len(_k_selection)):
        _color = _cmap(_gs_col_norm(_k_fine[_k_selection[_i]]))
        _ax_sk_meta_l.errorbar(_delta_Ps_liq, 
                               abs(_G) * _all_sk_spin_l_ave_3b[:,_k_selection[_i]], 
                               yerr=abs(_G) * _all_sk_spin_l_err_3b[:,_k_selection[_i]], 
                               linestyle='none', marker='o', color=_color)

        _ax_sk_meta_l.plot(_delta_Ps_liq, _Sk_fig_3b_hat_data_match[:,_k_selection[_i]], color=_color)
        _ax_sk_meta_l.plot(_delta_Ps_liq, _Sk_fig_3b_k2_hat_data_match[:,_k_selection[_i]], color=_color, linestyle='--')        

    _ax_sk_meta_l.plot(_delta_Ps_liq, 3.3e-12 * (_delta_Ps_liq ** -0.5), linestyle = '--', color = 'black', linewidth = 3)   

    SetAxPanelLabel(ax=_ax_sk_meta_l, label='$\\mathbf{\hat{S}(0) \sim (' + _pi_s_m_one + ')^{-1/2}}$', 
                    fs=_fonts['fs'], x_pos=0.395, y_pos=0.75, color='black')    
    
    '''
    k-Labels for the data
    '''
    index = 0
    _color = _cmap(_gs_col_norm(_k_fine[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_meta_l, label='$\mathbf{k\Delta x=' + ('%.2f' % _k_fine[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.61, color=_color, rotation=-19)

    index = 1
    _color = _cmap(_gs_col_norm(_k_fine[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_meta_l, label='$\mathbf{k\Delta x=' + ('%.2f' % _k_fine[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.47, color=_color, rotation=-4)
    
    index = 2
    _color = _cmap(_gs_col_norm(_k_fine[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_meta_l, label='$\mathbf{k\Delta x=' + ('%.2f' % _k_fine[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.29, color=_color)

    index = 3
    _color = _cmap(_gs_col_norm(_k_fine[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_meta_l, label='$\mathbf{k\Delta x\simeq \pi}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.075, color=_color)
        
######################################################


_ax_sk_crit_g = plt.subplot2grid((_ny_fig, _nx_fig), (1, 0), colspan = 1, rowspan = 1)
_panel_label="$(c)$"

_k_selection = (GSequence(n_steps=5, num=2, den=2) - 1)[:-1]
_k_selection = (GSequence(n_steps=8, num=2, den=2) - 1)[:-1]
_k_selection = (GSequence(n_steps=10, num=8, den=8) - 1)[::4]
_k_selection = (GSequence(n_steps=7, num=8, den=8) - 1)[::4]

_k_selection = [1, 7, 33, 254]

if True:
    _gs_col_norm = matplotlib.colors.Normalize(vmin = 0, vmax = np.pi)
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["r","b"])
    
    _every = 2
    _delta_G_gas = (abs(1/_G_c) - abs(1/figure_3c_G_list)) / abs(1/_G_c)

    _ax_sk_crit_g.set_xscale('log')
    _ax_sk_crit_g.set_yscale('log')
    
    _ax_sk_crit_g.set_xlim([_delta_G_gas[-1], _delta_G_gas[0]])

    SetAxTicksFont(_ax_sk_crit_g, _fonts['fs'])
    _ax_sk_crit_g.set_xlabel('$1-\hat{T}$', fontsize = _fonts['fs'])
    _ax_sk_crit_g.set_ylabel('$\hat{S}(k\Delta x)$', fontsize = _fonts['fs'])
    SetAxPanelLabel(ax=_ax_sk_crit_g, label=_panel_label, fs=_fonts['fs'], pos='ur')
    _ax_sk_crit_g.set_ylim([1e-13, 1e-7])

    for _i in [0, _Sk_fig_3c.shape[1] - 1]:
        _linestyle = '-.' if _i == 0 else '-'
        _linewidth = 3 if _i == 0 else 1        
        _color = _cmap(_gs_col_norm(_k_fine_alt_512[_i]))
        _ax_sk_crit_g.plot(_delta_G_gas, _Sk_fig_3c_hat[:,_i], color = _color, 
                           linestyle=_linestyle, linewidth=_linewidth)

    for _i in range(1, _Sk_fig_3c.shape[1] - 1):
        _color = _cmap(_gs_col_norm(_k_fine_alt_512[_i]))
        _ax_sk_crit_g.plot(_delta_G_gas, _Sk_fig_3c_hat[:,_i], color = _color, alpha = _alpha_crit)
    
    for _i in range(len(_k_selection)):
        _color = _cmap(_gs_col_norm(_k_fine_512[_k_selection[_i]]))

        _ax_sk_crit_g.errorbar(_delta_G_gas, 
                               np.abs(_G_fine[::2]) * _all_sk_bin_g_ave_3c[:,_k_selection[_i]], 
                               yerr=np.abs(_G_fine[::2]) * _all_sk_bin_g_err_3c[:,_k_selection[_i]], 
                               linestyle='none', marker='o', color=_color)

        _ax_sk_crit_g.plot(_delta_G_gas, _Sk_fig_3c_hat_data_match[:,_k_selection[_i]], color=_color)
        _ax_sk_crit_g.plot(_delta_G_gas, _Sk_fig_3c_k2_hat_data_match[:,_k_selection[_i]], color=_color, linestyle='--')        
    
    _ax_sk_crit_g.plot(_delta_G_gas, 5e-13 * (_delta_G_gas ** -1), linestyle = '--', color = 'black', linewidth = 3)

    '''
    k-Labels for the data
    '''
    index = 0
    _color = _cmap(_gs_col_norm(_k_fine_512[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_crit_g, label='$\mathbf{k\Delta x=' + ('%.2f' % _k_fine_512[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.007, y_pos=0.57, color=_color, rotation=-13)
    
    index = 1
    _color = _cmap(_gs_col_norm(_k_fine_512[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_crit_g, label='$\mathbf{k\Delta x=' + ('%.2f' % _k_fine_512[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.012, y_pos=0.47, color=_color, rotation=-1)

    index = 2
    _color = _cmap(_gs_col_norm(_k_fine_512[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_crit_g, label='$\mathbf{k\Delta x=' + ('%.2f' % _k_fine_512[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.275, color=_color)

    index = 3
    _color = _cmap(_gs_col_norm(_k_fine_512[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_crit_g, label='$\mathbf{k\Delta x\simeq \pi}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.025, color=_color)

    SetAxPanelLabel(ax=_ax_sk_crit_g, label='$\\mathbf{\hat{S}(0) \sim (1-\hat{T})^{-1}}$', 
                    fs=_fonts['fs'], x_pos=0.41, y_pos=0.67, color='black')    

######################################################


_ax_sk_crit_l = plt.subplot2grid((_ny_fig, _nx_fig), (1, 1), colspan = 1, rowspan = 1)
_panel_label="$(d)$"

if True:
    _gs_col_norm = matplotlib.colors.Normalize(vmin = 0, vmax = np.pi)
    _cmap = matplotlib.colors.LinearSegmentedColormap.from_list("MyCmapName",["r","b"])
    
    _every = 2
    _delta_G_liq = (abs(1/_G_c) - abs(1/figure_3d_G_list)) / abs(1/_G_c)

    _ax_sk_crit_l.set_xscale('log')
    _ax_sk_crit_l.set_yscale('log')
    _ax_sk_crit_l.set_xlim([_delta_G_liq[-1], _delta_G_liq[0]])
    _ax_sk_crit_l.set_ylim([1e-13, 1e-7])

    SetAxPanelLabel(ax=_ax_sk_crit_l, label=_panel_label, fs=_fonts['fs'], pos='ur')

    SetAxTicksFont(_ax_sk_crit_l, _fonts['fs'])
    SetAxPanelLabel(ax=_ax_sk_crit_l, label='$\\mathbf{\hat{S}(0) \sim (1-\hat{T})^{-1}}$', 
                    fs=_fonts['fs'], x_pos=0.4, y_pos=0.67, color='black')    
    
    _ax_sk_crit_l.set_xlabel('$1 - \hat{T}$', fontsize = _fonts['fs'])

    for _i in [0, _Sk_fig_3d_hat.shape[1] - 1]:
        _linestyle = '-.' if _i == 0 else '-'
        _linewidth = 3 if _i == 0 else 1        
        _color = _cmap(_gs_col_norm(_k_fine_alt_512[_i]))
        _ax_sk_crit_l.plot(_delta_G_liq, _Sk_fig_3d_hat[:,_i], color = _color, 
                           linestyle=_linestyle, linewidth=_linewidth)

    for _i in range(1, _Sk_fig_3d_hat.shape[1] - 1):
        _color = _cmap(_gs_col_norm(_k_fine_alt_512[_i]))
        _ax_sk_crit_l.plot(_delta_G_liq, _Sk_fig_3d_hat[:,_i], color = _color, alpha = _alpha_crit)

    for _i in range(len(_k_selection)):
        _color = _cmap(_gs_col_norm(_k_fine_512[_k_selection[_i]]))
        _ax_sk_crit_l.errorbar(_delta_G_liq, 
                               np.abs(_G_fine[::2]) * _all_sk_bin_l_ave_3d[:,_k_selection[_i]], 
                               yerr=np.abs(_G_fine[::2]) * _all_sk_bin_l_err_3d[:,_k_selection[_i]], 
                               linestyle='none', marker='o', color=_color)

        _ax_sk_crit_l.plot(_delta_G_liq, _Sk_fig_3d_hat_data_match[:,_k_selection[_i]], color=_color)
        _ax_sk_crit_l.plot(_delta_G_liq, _Sk_fig_3d_k2_hat_data_match[:,_k_selection[_i]], color=_color, linestyle='--')        

    _ax_sk_crit_l.plot(_delta_G_liq, 5e-13 * (_delta_G_liq ** -1), linestyle = '--', color = 'black', linewidth = 3)

    '''
    k-Labels for the data
    '''
    index = 0
    _color = _cmap(_gs_col_norm(_k_fine_512[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_crit_l, label='$\mathbf{k\Delta x=' + ('%.2f' % _k_fine_512[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.57, color=_color, rotation=-14)
    
    index = 1
    _color = _cmap(_gs_col_norm(_k_fine_512[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_crit_l, label='$\mathbf{k\Delta x=' + ('%.2f' % _k_fine_512[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.47, color=_color, rotation=-1)

    index = 2
    _color = _cmap(_gs_col_norm(_k_fine_512[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_crit_l, label='$\mathbf{k\Delta x=' + ('%.2f' % _k_fine_512[_k_selection[index]]) + '}$', 
                    fs=_fonts['fs'], x_pos=0.015, y_pos=0.28, color=_color)
    
    index = 3
    _color = _cmap(_gs_col_norm(_k_fine_512[_k_selection[index]]))    
    SetAxPanelLabel(ax=_ax_sk_crit_l, label='$\mathbf{k \Delta x\simeq \pi}$', 
                    fs=21, x_pos=0.015, y_pos=0.02, color=_color)    
        
plt.savefig(reproduced_figures / ('figure_3.png'), bbox_inches = 'tight', dpi = 300)
plt.savefig(reproduced_figures / ('figure_3.pdf'), bbox_inches = 'tight', dpi = 300)    

## Figure 4

In [ ]:
# Simulation/Data reading

import sympy as sp

from idpy.LBM.MultiPhase import ShanChenMultiPhase
from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP
from idpy.IdpyStencils.IdpyStencils import IDStencils

_f_stencil = IDStencils['LBM']['SC_D1Q2']
_xi_stencil = IDStencils['LBM']['XI_D1Q3']

from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP

_L, _tau, _G = 100, 1, -2.7

n = sp.Symbol('n')
psi_sym = sp.exp(-1/n)
psi_code = 'exp((NType)(-1./ln_0))'

from idpy.Utils.SimpleTiming import SimpleTiming
from idpy.Utils.ManageData import ManageData

st = SimpleTiming()
md = ManageData(dump_file='FlatInterface.json')
read_flag = md.Read(kind='json')

print("Checking if the data is already there...", end="")
if not md.IsThereKey('density_profile', kind='json'):
    print("no!")
    print("Running the simulation")
    # Declaring simulation object _test_meta
    flat_interface_sim = \
        ShanChenMultiPhase(dim_sizes = (_L,) * dimension, 
                           xi_stencil = _xi_stencil, 
                           f_stencil = _f_stencil,
                           psi_code = psi_code, 
                           psi_sym = psi_sym, 
                           e2_val = 1, 
                           SC_G = _G, tau = _tau,
                           lang = preferred_lang, device = preferred_device, 
                           cl_kind = preferred_kind, 
                           optimizer_flag = True)


    n_g = GetDensitySpinodalDistance(0, psi_sym, _G, 'n_g')
    n_l = GetDensitySpinodalDistance(0, psi_sym, _G, 'n_l')

    flat_interface_sim.InitFlatInterface(n_g = n_g, n_l = n_l, width = _L//2, direction = 0)
        
    st.Start()
    flat_interface_sim.MainLoop(
        time_steps = range(*_time_tuple), 
        convergence_functions = [CheckUConvergenceSCMP],
        profiling = False
    )
    st.End()

    data_set = {'n_field': list(flat_interface_sim.GetDensityField())}
    md.PushData(data=data_set, key='density_profile')
    md.Dump(kind='json', indent=4)
    
    flat_interface_sim.End()
else:
    print("yes!")
    print("Reading the flat interface profile from the file")
    data_set = md.PullData(key='density_profile')

n_flat_profile = np.array(data_set['n_field'])

In [ ]:
# Plotting Figure 4

import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import scipy.interpolate as interpolate
from idpy.Utils.Plots import SetAxPanelLabelCoords, SetAxPanelLabel

from idpy.Utils.Plots import SetAxPanelLabelCoords
from idpy.Utils.Plots import CreateFiguresPanels, SetMatplotlibLatexParamas
from idpy.Utils.Plots import SetDefaultFonts, SetAxTicksFont, SetAxPanelLabel

from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

SetMatplotlibLatexParamas([rc], [rcParams])
_fonts = SetDefaultFonts([rc], )

psi_f = sp.lambdify(n, psi_sym)
psi_swap = psi_f(n_flat_profile)

psi_swap_m1 = np.append(psi_swap[-1], psi_swap[:-1])
psi_swap_p1 = np.append(psi_swap[1:], psi_swap[0])
psi_swap_m2 = np.append(psi_swap_m1[-1], psi_swap_m1[:-1])
psi_swap_p2 = np.append(psi_swap_p1[1:], psi_swap_p1[0])

p_b = \
    n_flat_profile / 3 + _G * (psi_swap ** 2)/2

p_n = \
    n_flat_profile / 3 + \
    _G * psi_swap * (psi_swap_m1 + psi_swap_p1) / 4

_ratio = 0.8
fig = CreateFiguresPanels(_nx=1,_ny=2, _x_size=6.75/_ratio, _y_size=2.95/_ratio)

ax_0 = plt.subplot2grid((2, 1), (0, 0), colspan = 1, rowspan = 1)
ax_0.plot(p_b)
ax_0.axhline(y=p_b[0], linestyle='--', color='black', label="$P_0$")
#3 ax_0.set_xlabel('$x$', fontsize = _fonts['fs'])
ax_0.set_ylabel('$P_{\\mbox{\\small{b}}}$', fontsize = _fonts['fs'])
SetAxTicksFont(ax_0, _fonts['fs'])
SetAxPanelLabel(ax=ax_0, label="$(a)$", fs=_fonts['fs'], x_pos=0.925, y_pos=0.075)
## SetAxPanelLabelCoords(ax=ax_0, label="$P_0$", fs=_fonts['fs'], x_pos=107, y_pos=_p_b[0] * (1 - 0.0025))
lgnd_points = ax_0.legend(frameon = False, loc = 'upper right')

ax_1 = plt.subplot2grid((2, 1), (1, 0), colspan = 1, rowspan = 1)
ax_1.plot(p_n, color='darkgreen', label='$P_{\\mbox{\\small{N}}}(x) = n(x) T - \\frac{1}{4}\\psi(x)[\\psi(x+1)+\\psi(x-1)] = P_0$')
## 
ax_1.set_ylim([np.amin(p_n) * (1 - 0.5e-13), np.amax(p_n) * (1 + 4e-13)])


lgnd_points = ax_1.legend(frameon = False, loc = 'upper right')

ax_1.set_xlabel('$x$', fontsize = _fonts['fs'])
ax_1.set_ylabel('$P_{\\mbox{\\small{N}}}$', fontsize = _fonts['fs'])
SetAxPanelLabel(ax=ax_1, label="$(b)$", fs=_fonts['fs'], x_pos=0.925, y_pos=0.075)

SetAxTicksFont(ax_1, _fonts['fs'])

plt.savefig(reproduced_figures / ('figure_4.pdf'), bbox_inches = 'tight', dpi = 300)
plt.savefig(reproduced_figures / ('figure_4.png'), bbox_inches = 'tight', dpi = 300)

## Figure 5
### Symbolic Calculations

In [ ]:
# Preparing the Lattice Pressure Tensor coefficients

import sympy as sp
'''
Defining the weights symbols 
'''
_w_sym, _l2_list_E12 = {}, [1, 2, 4, 5, 8, 9, 10, 13, 16, 17]
for _l2 in _l2_list_E12:
    _w_sym[_l2] = sp.Symbol("w(" + str(_l2) + ")")

'''
Defining the coefficients symbols and weights combinations
'''
'''
a_xx coefficients
'''
_a_N_m40p4 = sp.Symbol("a^{(N)}_{[-4, 0, 4]}")
_a_N_m30p3 = sp.Symbol("a^{(N)}_{[-3, 0, 3]}")
_a_N_m20p2 = sp.Symbol("a^{(N)}_{[-2, 0, 2]}")
_a_N_m10p1 = sp.Symbol("a^{(N)}_{[-1, 0, 1]}")

_a_N_m40p4_w = 2 * _w_sym[16] + 4 * _w_sym[17]
_a_N_m30p3_w = 3 * _w_sym[9] / 2 + 3 * _w_sym[10] + 3 * _w_sym[13]
_a_N_m20p2_w = \
    _w_sym[4] + 2 * _w_sym[5] + 2 * _w_sym[8] + \
    2 * _w_sym[13]
_a_N_m10p1_w = \
    _w_sym[1] / 2 + _w_sym[2] + _w_sym[5] + \
    _w_sym[10] + _w_sym[17]

_a_coeff_N = [_a_N_m40p4, _a_N_m30p3, _a_N_m20p2, _a_N_m10p1]
_a_coeff_N_w = [_a_N_m40p4_w, _a_N_m30p3_w, _a_N_m20p2_w, _a_N_m10p1_w]

'''
a_yy coefficients
'''
_a_T_m40p4 = sp.Symbol("a^{(T)}_{[-4, 0, 4]}")
_a_T_m30p3 = sp.Symbol("a^{(T)}_{[-3, 0, 3]}")
_a_T_m20p2 = sp.Symbol("a^{(T)}_{[-2, 0, 2]}")
_a_T_m10p1 = sp.Symbol("a^{(T)}_{[-1, 0, 1]}")
_a_T_0 = sp.Symbol("a^{(T)}_{[0]}")

_a_T_m40p4_w = _w_sym[17] / 4
_a_T_m30p3_w = _w_sym[10] / 3 + 4 * _w_sym[13] / 3
_a_T_m20p2_w = _w_sym[5] / 2 + 2 * _w_sym[8] + 9 * _w_sym[13] / 2
_a_T_m10p1_w = \
    _w_sym[2] + 4 * _w_sym[5] + 9 * _w_sym[10] + \
    16 * _w_sym[17]
_a_T_0_w = \
    _w_sym[1] + 4 * _w_sym[4] + 9 * _w_sym[9] + \
    16 * _w_sym[16]

_a_coeff_T = [_a_T_m40p4, _a_T_m30p3, _a_T_m20p2, _a_T_m10p1, _a_T_0]
_a_coeff_T_w = [_a_T_m40p4_w, _a_T_m30p3_w, _a_T_m20p2_w, _a_T_m10p1_w, _a_T_0_w]

'''
b_xx coefficients
'''
_b_N_22 = sp.Symbol("b^{(N)}_{[2,2]}")
_b_N_11 = sp.Symbol("b^{(N)}_{[1,1]}")
_b_N_13 = sp.Symbol("b^{(N)}_{[1,3]}")
_b_N_12 = sp.Symbol("b^{(N)}_{[1,2]}")

_b_N_22_w = 4 * _w_sym[16] + 8 * _w_sym[17]
_b_N_11_w = \
    2 * _w_sym[4] + 4 * _w_sym[5] + \
    4 * _w_sym[8] + 4 * _w_sym[13]

_b_N_13_w = 4 * _w_sym[16] + 8 * _w_sym[17]
_b_N_12_w = 3 * _w_sym[9] + 6 * _w_sym[10] + 6 * _w_sym[13]


_b_coeff_N = [_b_N_22, _b_N_11, _b_N_13, _b_N_12]
_b_coeff_N_w = [_b_N_22_w, _b_N_11_w, _b_N_13_w, _b_N_12_w]

'''
b_yy coefficients
'''
_b_T_22 = sp.Symbol("b^{(T)}_{[2,2]}")
_b_T_11 = sp.Symbol("b^{(T)}_{[1,1]}")
_b_T_13 = sp.Symbol("b^{(T)}_{[1,3]}")
_b_T_12 = sp.Symbol("b^{(T)}_{[1,2]}")

_b_T_22_w = _w_sym[17] / 2
_b_T_11_w = _w_sym[5] + 4 * _w_sym[8] + 9 * _w_sym[13]
_b_T_13_w = _w_sym[17] / 2
_b_T_12_w = 2 * _w_sym[10] / 3 + 8 * _w_sym[13] / 3

_b_coeff_T = [_b_T_22, _b_T_11, _b_T_13, _b_T_12]
_b_coeff_T_w = [_b_T_22_w, _b_T_11_w, _b_T_13_w, _b_T_12_w]

_G, _c_s2 = sp.symbols("G c_s^2")
_psi, _x = sp.Function('\psi'), sp.Symbol('x')

_P_N = \
    _G * _c_s2 * _a_N_m40p4 * _psi(_x) * (_psi(_x + 4) + _psi(_x - 4)) + \
    _G * _c_s2 * _a_N_m30p3 * _psi(_x) * (_psi(_x + 3) + _psi(_x - 3)) + \
    _G * _c_s2 * _a_N_m20p2 * _psi(_x) * (_psi(_x + 2) + _psi(_x - 2)) + \
    _G * _c_s2 * _a_N_m10p1 * _psi(_x) * (_psi(_x + 1) + _psi(_x - 1)) + \
    _G * _c_s2 * _b_N_22 * _psi(_x + 2) * _psi(_x - 2) + \
    _G * _c_s2 * _b_N_11 * _psi(_x + 1) * _psi(_x - 1) + \
    _G * _c_s2 * _b_N_13 * (_psi(_x - 1) * _psi(_x + 3) + _psi(_x + 1) * _psi(_x - 3)) + \
    _G * _c_s2 * _b_N_12 * (_psi(_x - 1) * _psi(_x + 2) + _psi(_x + 1) * _psi(_x - 2))

_D2E12 = SCFStencils(E = BasisVectors(x_max = 4), 
                         len_2s = [1, 2, 4, 5, 8, 9, 10, 13, 16, 17])

_D2E12.GetWolfEqs()
_D2E12.GetTypEqs()

I40 = sp.Symbol('I_{4,0}')
I60 = sp.Symbol('I_{6,0}')
I80 = sp.Symbol('I_{8,0}')
I81 = sp.Symbol('I_{8,1}')
I100 = sp.Symbol('I_{10,0}')
I101 = sp.Symbol('I_{10,1}')

_w_sym_list = [_w_sym[key] for key in _w_sym]

_eq_s = \
    [_D2E12.e_sym[2] - _D2E12.e_expr[2], 
     _D2E12.e_sym[4] - _D2E12.e_expr[4], 
     _D2E12.e_sym[6] - _D2E12.e_expr[6], 
     _D2E12.e_sym[8] - _D2E12.e_expr[8], 
     _D2E12.e_sym[10] - _D2E12.e_expr[10], 
     _D2E12.e_sym[12] - _D2E12.e_expr[12], 
     I40 - _D2E12.typ_eq_s[4][0], 
     I60 - _D2E12.typ_eq_s[6][0], 
     I100 - _D2E12.typ_eq_s[10][0], 
     I101 - _D2E12.typ_eq_s[10][1]]

_w_EI_sol_dict = sp.solve(_eq_s, _w_sym_list)

G_sym, cs2_sym = sp.Symbol('G'), sp.Symbol('c_s^2')
psi_0_sym, psip_0_sym = sp.Symbol('\psi_0'), sp.Symbol("\psi'_0")

'''
Need to change the names here! hat <-> not hat !!! So that name and notation match with the paper
'''

c0 = _a_N_m10p1_w + _a_N_m20p2_w + _a_N_m30p3_w + _a_N_m40p4_w
c1 = _a_N_m10p1_w + _b_N_11_w + _b_N_12_w + _b_N_13_w
c2 = _a_N_m20p2_w + _b_N_12_w + _b_N_22_w
c3 = _a_N_m30p3_w + _b_N_13_w
c4 = _a_N_m40p4_w

c0_hat = 2*G_sym*cs2_sym*c0*psi_0_sym*psip_0_sym
c1_hat = 2*G_sym*cs2_sym*c1*psi_0_sym*psip_0_sym
c2_hat = 2*G_sym*cs2_sym*c2*psi_0_sym*psip_0_sym
c3_hat = 2*G_sym*cs2_sym*c3*psi_0_sym*psip_0_sym
c4_hat = 2*G_sym*cs2_sym*c4*psi_0_sym*psip_0_sym

c0b, c1b, c2b, c3b, c4b = c0_hat / cs2_sym, c1_hat / c0_hat, c2_hat / c0_hat, c3_hat / c0_hat, c4_hat / c0_hat

c1b = sp.simplify(c1b)
c2b = sp.simplify(c2b)
c3b = sp.simplify(c3b)
c4b = sp.simplify(c4b)

c1b_sol, c2b_sol, c3b_sol, c4b_sol = c1b, c2b, c3b, c4b
c0_sol, c1_sol, c2_sol, c3_sol, c4_sol = \
    c0, c1, c2, c3, c4

for w_sym in _w_EI_sol_dict:
    c1b_sol = c1b_sol.subs(w_sym, _w_EI_sol_dict[w_sym])
    c2b_sol = c2b_sol.subs(w_sym, _w_EI_sol_dict[w_sym])
    c3b_sol = c3b_sol.subs(w_sym, _w_EI_sol_dict[w_sym])
    c4b_sol = c4b_sol.subs(w_sym, _w_EI_sol_dict[w_sym])
    
    c0_sol = c0_sol.subs(w_sym, _w_EI_sol_dict[w_sym])
    c1_sol = c1_sol.subs(w_sym, _w_EI_sol_dict[w_sym])
    c2_sol = c2_sol.subs(w_sym, _w_EI_sol_dict[w_sym])
    c3_sol = c3_sol.subs(w_sym, _w_EI_sol_dict[w_sym])
    c4_sol = c4_sol.subs(w_sym, _w_EI_sol_dict[w_sym])    
    
c1b_sol = sp.simplify(c1b_sol)
c2b_sol = sp.simplify(c2b_sol)
c3b_sol = sp.simplify(c3b_sol)
c4b_sol = sp.simplify(c4b_sol)

c0_sol = sp.simplify(c0_sol)
c1_sol = sp.simplify(c1_sol)
c2_sol = sp.simplify(c2_sol)
c3_sol = sp.simplify(c3_sol)
c4_sol = sp.simplify(c4_sol)

### One-dimensional Stencil

In [ ]:
# Solving the equation to determine the stencil weights in two dimensions, then projecting in 1d

_D2E12_num = SCFStencils(E = BasisVectors(x_max = 4), 
                         len_2s = [1, 2, 4, 5, 8, 9, 10, 13, 16, 17])

_D2E12_num.GetWolfEqs()
_D2E12_num.GetTypEqs()

eps_val = sp.Rational(-1, 2)
e4_eps_val = sp.Rational(2 + eps_val, 6 * (1 - eps_val))

_eq_s_w = \
    [_D2E12_num.e_expr[2] - 1, 
     c3b - sp.Rational(738, 46349) * sp.Rational(2 ** 6),
     c2b - sp.Rational(13710, 46349) * sp.Rational(2 ** 3),
     c1b - sp.Rational(59322, 46349) * sp.Rational(50, 28),
     _D2E12_num.typ_eq_s[4][0], 
     _D2E12_num.typ_eq_s[6][0], 
     _D2E12_num.typ_eq_s[8][0], 
     _D2E12_num.typ_eq_s[8][1],
     _D2E12_num.typ_eq_s[10][0], 
     _D2E12_num.typ_eq_s[10][1]
    ]

print("Two-dimensional forcing weights:")
print(_D2E12_num.FindWeights(_eq_s_w))

_stencil_dict, _w_swap = _D2E12_num.PushStencil(), {}
_ex_list = []
for e_i, e in enumerate(_stencil_dict['Es']):
    _ex_list += [e[0]]
_ex_list = np.array(_ex_list)
_ex_max, _ex_min = np.amax(_ex_list), np.amin(_ex_list)

_x_list = list(np.arange(_ex_min, _ex_max + 1))
for x in _x_list:
    _w_swap[x] = 0
    
for e_i, e in enumerate(_stencil_dict['Es']):
    _w_swap[e[0]] += _stencil_dict['Ws'][e_i]

_adapted_dict = {'XIs': tuple(), 'Ws': tuple(), 'e2': 1}
for x in _x_list:
    if x != 0:
        _adapted_dict['XIs'] += ((x,),)
        _adapted_dict['Ws'] += (_w_swap[x],)

print("One-dimensional forcing weights:")
_adapted_dict['Ws']

### Simulations

In [ ]:
# Running the simulations and dumping results/reading results

G_sym, cs2_sym = sp.Symbol('G'), sp.Symbol('c_s^2')
##psi_0_sym, psip_0_sym = sp.Symbol('\psi_0'), sp.Symbol("\psi'_0")

n, a_sym, b_sym, RT_sym = \
    sp.Symbol('n', positive=True, real=True), \
    sp.Symbol('a', positive=True, real=True), \
    sp.Symbol('b', positive=True, real=True), \
    sp.Symbol('RT', positive=True, real=True)

n_c, RT_c, P_c = sp.Symbol('n_c'), sp.Symbol('RT_c'), sp.Symbol('P_c')

P_CS = \
    RT_sym * n * (1 + (b_sym * n / 4) + (b_sym * n / 4) ** 2 - (b_sym * n / 4) ** 3) / ((1 - (b_sym * n / 4)) ** 3) - \
    a_sym * (n ** 2)

cond_1 = sp.simplify(((b_sym * n - 4) ** 4) * sp.simplify(sp.diff(P_CS, n, 1)))
cond_2 = sp.simplify(((b_sym * n - 4) ** 5) * sp.diff(P_CS, n, 2))

solutions_cpt = sp.nonlinsolve([cond_1, cond_2], (a_sym, b_sym))

all_solutions = []
for _ in solutions_cpt:
    all_solutions += [_]

a_sol_c = all_solutions[-1][0].evalf().subs(n, n_c).subs(RT_sym, RT_c)
b_sol_c = all_solutions[-1][1].evalf().subs(n, n_c).subs(RT_sym, RT_c)

P_c_val = P_CS.subs(a_sym, a_sol_c).subs(b_sym, b_sol_c).subs(n, n_c).subs(RT_sym, RT_c)
P_c_SC = n_c * (1 - 1 / (2 * sp.log(2))) / 3
n_c_SC = sp.log(2)

RT_c_val = sp.solve(P_c_SC - P_c_val, RT_c)[0]

'''
Need to loop here
'''
from idpy.Utils.ManageData import ManageData
md = ManageData(dump_file='Carnahan-Starling-ModulatedSK.json')
read_flag = md.Read(kind='json')

tuned_sk_ratios = [84/100, 82/100, 80/100, 77/100]
tuned_sk_keys = ['84/100', '82/100', '80/100', '77/100']

for seed, t in enumerate(zip(tuned_sk_ratios, tuned_sk_keys)):
    ratio, key = t[0], t[1]

    RT_val = RT_c_val * ratio ## [84/100, 82/100, 80/100, 77/100]

    print("Checking if the data is already there...", end="")
    if not md.IsThereKey(key, kind='json'):
        print("no!")
        print("Running the simulation")        
        P_CS_f = P_CS.subs(a_sym, a_sol_c).subs(b_sym, b_sol_c).subs(n_c, n_c_SC).subs(RT_c, RT_c_val).subs(RT_sym, RT_val)
        ## P_CS_lam = lambda x: P_CS_f.subs(n, x)
        
        
        '''
        Computing the expression for psi and its 'code' version
        '''
        
        psi_sym = sp.sqrt(2 * (P_CS_f - cs2_sym * n) / (G_sym)).subs(G_sym, -1).subs(cs2_sym, 1/3)
        
        def FindMatchingBrackets(chars):
            d, start, counter = {}, [], 0
            for i, c in enumerate(chars):
                if c == '(':
                    start.append(i)
                if c == ')':
                    d[counter] = (start.pop(), i)
                    counter += 1
            return d
        
        psi_CS_code_sp = str(psi_sym.subs(n, sp.Symbol('ln_0')).evalf())
        psi_CS_code_sp = psi_CS_code_sp.replace('ln_0**2', 'ln_0*ln_0').replace('ln_0**3', 'ln_0*ln_0*ln_0')
        psi_CS_code_sp = psi_CS_code_sp.replace('(1.0 - 0.188190744838748*ln_0)**3', '(' + ('(1.0 - 0.188190744838748*ln_0)*' * 3)[:-1] + ')')
        
        rnd_brks = FindMatchingBrackets(psi_CS_code_sp)
        highest_level = list(rnd_brks.keys())[-1]
        last_brks = rnd_brks[highest_level]
        psi_CS_code_sp = psi_CS_code_sp[:last_brks[0]] + 'sqrt((NType)(' + psi_CS_code_sp[last_brks[0] + 1:last_brks[1]] + '))'
        
        '''
        Checking the equilibrium values for the densities using the simplest stencil
        '''
        from idpy.LBM.MultiPhase import ShanChenMultiPhase
        from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP
        from idpy.IdpyStencils.IdpyStencils import IDStencils
        
        _L, _G = 96, -1
        _kBT = 1e-10
        _c_s2, _tau = 1/3, 1
        
        _f_stencil = IDStencils['LBM']['SC_D1Q2']
        _xi_stencil = IDStencils['LBM']['XI_D1Q3']
        
        one_dimensional_sim = \
            ShanChenMultiPhase(dim_sizes = (_L,), 
                               xi_stencil = _xi_stencil, 
                               f_stencil = _f_stencil,
                               psi_code = psi_CS_code_sp, 
                               psi_sym = psi_sym, 
                               e2_val = 1, 
                               SC_G = _G, tau = _tau,
                               lang = preferred_lang, device = preferred_device, 
                               cl_kind = preferred_kind, 
                               optimizer_flag = True, 
                               fluctuations = 'Gross2011', 
                               prng_distribution = 'gaussian', 
                               indep_gaussian = False, 
                               prng_kind = 'MMIX',
                               prng_init_from = 'numpy', 
                               init_seed = seed + 1)
        
        one_dimensional_sim.InitFlatInterface(n_g=0.1, 
                                              n_l=1.5, width=_L//2, direction=0)
        
        
        from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP
        
        one_dimensional_sim.MainLoop(
            time_steps = range(0, 2 ** 22 + 1, 2 ** 14), 
            convergence_functions = [CheckUConvergenceSCMP],
            profiling = False)
        
        n_field = one_dimensional_sim.GetDensityField()
        n_g_cs, n_l_cs = np.amin(n_field), np.amax(n_field)
    
        one_dimensional_sim.End()
    
        ##############################################################
        '''
        checking the liquid denisty value with the more sophisticated stencil
        '''
    
        from idpy.LBM.MultiPhase import ShanChenMultiPhase
        from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP
        from idpy.IdpyStencils.IdpyStencils import IDStencils
        
        _L, _G = 96, -1
        _kBT = 1e-10
        _c_s2, _tau = 1/3, 1
        
        _f_stencil = _adapted_dict
        _xi_stencil = IDStencils['LBM']['XI_D1Q3']
        
        _n_start=n_l_cs
        
        one_dimensional_sim = \
            ShanChenMultiPhase(dim_sizes = (_L,), 
                               xi_stencil = _xi_stencil, 
                               f_stencil = _f_stencil,
                               psi_code = psi_CS_code_sp, 
                               psi_sym = psi_sym, 
                               e2_val = 1, 
                               SC_G = _G, tau = _tau,
                               lang = preferred_lang, device = preferred_device, 
                               cl_kind = preferred_kind, 
                               optimizer_flag = True, 
                               fluctuations = 'Gross2011', 
                               prng_distribution = 'gaussian', 
                               indep_gaussian = False, 
                               prng_kind = 'MMIX',
                               prng_init_from = 'numpy', 
                               init_seed = seed + 1)
        
        one_dimensional_sim.InitFlatInterface(n_g=_n_start, 
                                              n_l=_n_start, width=_L//2, direction=0)
        
        from idpy.LBM.MultiPhaseLoopChecks import CheckUConvergenceSCMP
        
        one_dimensional_sim.MainLoop(
            time_steps = range(0, 2 ** 22 + 1, 2 ** 14), 
            convergence_functions = [CheckUConvergenceSCMP],
            profiling = False)
        
        n_field = one_dimensional_sim.GetDensityField()
        print(np.amin(n_field), np.amax(n_field), np.amax(n_field)/np.amin(n_field))
    
        ########################################################
        '''
        Measuring the structure factors
        '''
    
        from idpy.Utils.SimpleTiming import SimpleTiming
        
        _st = SimpleTiming()
        
        _st.Start()
        one_dimensional_sim.MainLoopGross2011SRT(
            time_steps = range(0, 2 ** 26 + 1, 2 ** 11), 
            convergence_functions = [CheckUConvergenceSCMP, StructureFactors1D],
            convergence_functions_args=[{'print_flag': False}, {}],
            profiling = False, print_flag = False,
            kBT = _kBT, n0 = n_l_cs)
        _st.End()
        _st.PrintElapsedTime()
        
        plt.plot(one_dimensional_sim.GetDensityField())
        plt.show()
        plt.close()
        
        one_dimensional_sim.End()
        
        '''
        Data Analysis for the S(k)
        '''
        
        _n2_ft_data = np.array(one_dimensional_sim.sims_vars['n2_ft'])
        
        _n2_ft_ave, _n2_ft_err = [], []
        for _i in range(_n2_ft_data.shape[1]):
            _mean_swap, _err_swap = ComputeMeanErr(_n2_ft_data[:,_i])    
            _n2_ft_ave += [_mean_swap]
            _n2_ft_err += [_err_swap]    
        
        _n2_ft_ave, _n2_ft_err = np.array(_n2_ft_ave), np.array(_n2_ft_err)
        
        _k_range = one_dimensional_sim.sims_vars['nx_list'] * 2 * np.pi / _L
    
        ##############################################################
        '''
        Computing the coefficients for the theoretical structure factors
        '''
        e_sol, c_sol, c_hat_sol = {}, {}, {}
        
        _n_start = n_l_cs
        
        psi_0_val = psi_sym.subs(n, _n_start).evalf()
        psip_0_val = sp.diff(psi_sym, n).subs(n, _n_start).evalf()
        
        e_sol, c_sol, c_hat_sol = {}, {}, {}
        print("e_n")
        for e_i in _D2E12.e_expr:
            expr = _D2E12.e_expr[e_i]
            for w_i, w_s in enumerate([_w_sym[key] for key in _w_sym]):
                expr = expr.subs(w_s, _D2E12_num.w_sol[0][w_i])
            e_sol[e_i] = expr
            
        for e_i in e_sol:
            print(e_i, e_sol[e_i], float(e_sol[e_i]))
        print()
        
        print("c_bar_sol")
        for c_i, c_expr in enumerate([c0b, c1b, c2b, c3b, c4b]):
            for w_i, w_s in enumerate([_w_sym[key] for key in _w_sym]):
                c_expr = c_expr.subs(w_s, _D2E12_num.w_sol[0][w_i])
            c_sol[c_i] = c_expr.subs(psi_0_sym, psi_0_val).subs(psip_0_sym, psip_0_val).subs(G_sym, _G * 3)
            ##print(c_i, c_sol[c_i], float(c_sol[c_i]), psi_0_val * psip_0_val * G)
        print()
        
        def SkE12(k):
            value = 1 / (1 + c_sol[0] * 
                        (1. + 
                         c_sol[1] * np.cos(k) + c_sol[2] * np.cos(2 * k) + 
                         c_sol[3] * np.cos(3 * k) + c_sol[4] * np.cos(4 * k)))
            return value
    
        ################################################################
        '''
        Dumping data in the json file
        '''
    
        _k_range_fine = np.linspace(0, np.pi, 2 ** 10)
    
        data_set = {'k_fine': list(_k_range_fine), 
                    'ske12': [float(_) for _ in SkE12(_k_range_fine)],
                    'k_range': list(_k_range[1:]), 
                    'data_ave': list(_n2_ft_ave[1:] / float(_kBT * _n_start) / 3), 
                    'data_err': list(_n2_ft_err[1:] / float(_kBT * _n_start) / 3), 
                    'kBT': float(_kBT), 'n_start': float(_n_start)}
    
        md.PushData(data=data_set, key=key)
        md.Dump(kind='json', indent=4)
    else:
        print("yes!")

In [ ]:
# Reading the data from file

md = ManageData(dump_file='Carnahan-Starling-ModulatedSK.json')
read_flag = md.Read(kind='json')

tuned_sk_ratios = [84/100, 82/100, 80/100, 77/100]
tuned_sk_keys = ['84/100', '82/100', '80/100', '77/100']

tuned_sk_data = {}

for t in zip(tuned_sk_ratios, tuned_sk_keys):
    ratio, key = t[0], t[1]
    key_flag = md.IsThereKey(key, kind='json')
    if key_flag:
        tuned_sk_data[key] = md.PullData(key=key)

# Plotting Figure 5

import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import scipy.interpolate as interpolate
from idpy.Utils.Plots import SetAxPanelLabelCoords, SetMatplotlibLatexParamas, SetDefaultFonts
from idpy.Utils.Plots import CreateFiguresPanels, SetAxPanelLabel, SetAxTicksFont

from pathlib import Path

reproduced_figures = Path('./reproduced-figures')
if not reproduced_figures.is_dir():
    reproduced_figures.mkdir()

SetMatplotlibLatexParamas([rc], [rcParams])
_fonts = SetDefaultFonts([rc], font_size=16)

_nx_fig, _ny_fig = 1, 1
_y_size, _xy_ratio = 4.3, 1.4
fig = CreateFiguresPanels(_nx=_nx_fig,_ny=_ny_fig, _x_size=_y_size*_xy_ratio, _y_size=_y_size)

_ax_Sk = plt.subplot2grid((_ny_fig, _nx_fig), (0, 0), colspan = 1, rowspan = 1)

key_colors = {'84/100': 'purple', '82/100': 'darkgreen', 
              '80/100': 'blue', '77/100': 'red'}

key_values = {'84/100': 0.84, '82/100': 0.82, 
              '80/100': 0.8, '77/100': 0.77}

key_styles = {'84/100': '-', '82/100': '-', '80/100': '-', '77/100': '-'}

for key in tuned_sk_data:
    _ax_Sk.errorbar(tuned_sk_data[key]['k_range'], 
                    tuned_sk_data[key]['data_ave'], 
                    tuned_sk_data[key]['data_err'], color=key_colors[key])

    _k_range_fine = np.linspace(0, np.pi, 2 ** 10)
    _ax_Sk.plot(tuned_sk_data[key]['k_fine'], tuned_sk_data[key]['ske12'], 
                color=key_colors[key], linestyle=key_styles[key],
                label = '$\hat{T}=' + str(key_values[key]) + '$')

_ax_Sk.axhline(y=1, color='black', linestyle='--')
_ax_Sk.arrow(1, 1.25, 0., -0.2, length_includes_head=True, head_width=0.05, head_length=0.02, color='black')
SetAxPanelLabelCoords(_ax_Sk, '$\hat{T}$', fs=_fonts['l_fs'], x_pos = 0.82, y_pos = 1.14)


_ax_Sk.set_xlabel('$k\Delta x$', fontsize=_fonts['fs'])
_ax_Sk.set_ylabel('$S(k\Delta x) c_s^2 / k_B\\vartheta$', fontsize=_fonts['fs'])
SetAxTicksFont(_ax_Sk, _fonts['fs'])

_ax_Sk.legend()

plt.savefig(reproduced_figures / ('figure_5.pdf'), dpi=300, bbox_inches='tight')
plt.savefig(reproduced_figures / ('figure_5.png'), dpi=300, bbox_inches='tight')